In [ ]:
!pip install imgviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 58.0 MB/s eta 0:00:00


In [ ]:
!nvidia-smi

Fri Sep 22 15:21:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    23W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!wget https://www.dropbox.com/sh/9yu1elddll00sdl/AAC-rSJdLX0C6HhKXGKMOIija
!unzip /content/AAC-rSJdLX0C6HhKXGKMOIija
!unzip /content/Replica_Dataset/office_0.zip
!unzip /content/Replica_Dataset/semantic_info.zip

Output streaming troncato alle ultime 5000 righe.
  inflating: office_0/Sequence_1/semantic_class/semantic_class_381.png  
  inflating: office_0/Sequence_1/semantic_class/vis_sem_class_382.png  
  inflating: office_0/Sequence_1/semantic_class/semantic_class_383.png  
  inflating: office_0/Sequence_1/semantic_class/vis_sem_class_384.png  
  inflating: office_0/Sequence_1/semantic_class/semantic_class_385.png  
  inflating: office_0/Sequence_1/semantic_class/vis_sem_class_385.png  
  inflating: office_0/Sequence_1/semantic_class/vis_sem_class_386.png  
  inflating: office_0/Sequence_1/semantic_class/semantic_class_387.png  
  inflating: office_0/Sequence_1/semantic_class/semantic_class_390.png  
  inflating: office_0/Sequence_1/semantic_class/vis_sem_class_390.png  
  inflating: office_0/Sequence_1/semantic_class/semantic_class_391.png  
  inflating: office_0/Sequence_1/semantic_class/vis_sem_class_391.png  
  inflating: office_0/Sequence_1/semantic_class/semantic_class_392.png  
  infla

In [ ]:
!unzip /content/Replica_Dataset/office_4.zip

Output streaming troncato alle ultime 5000 righe.
  inflating: office_4/Sequence_1/semantic_class/semantic_class_366.png  
  inflating: office_4/Sequence_1/semantic_class/semantic_class_367.png  
  inflating: office_4/Sequence_1/semantic_class/vis_sem_class_367.png  
  inflating: office_4/Sequence_1/semantic_class/vis_sem_class_368.png  
  inflating: office_4/Sequence_1/semantic_class/vis_sem_class_369.png  
  inflating: office_4/Sequence_1/semantic_class/semantic_class_370.png  
  inflating: office_4/Sequence_1/semantic_class/vis_sem_class_371.png  
  inflating: office_4/Sequence_1/semantic_class/semantic_class_372.png  
  inflating: office_4/Sequence_1/semantic_class/vis_sem_class_372.png  
  inflating: office_4/Sequence_1/semantic_class/semantic_class_373.png  
  inflating: office_4/Sequence_1/semantic_class/vis_sem_class_373.png  
  inflating: office_4/Sequence_1/semantic_class/vis_sem_class_376.png  
  inflating: office_4/Sequence_1/semantic_class/semantic_class_377.png  
  inflat

In [ ]:
!unzip /content/Replica_Dataset/office_2.zip

Output streaming troncato alle ultime 5000 righe.
  inflating: office_2/Sequence_1/semantic_class/semantic_class_366.png  
  inflating: office_2/Sequence_1/semantic_class/semantic_class_367.png  
  inflating: office_2/Sequence_1/semantic_class/vis_sem_class_367.png  
  inflating: office_2/Sequence_1/semantic_class/vis_sem_class_368.png  
  inflating: office_2/Sequence_1/semantic_class/vis_sem_class_369.png  
  inflating: office_2/Sequence_1/semantic_class/semantic_class_370.png  
  inflating: office_2/Sequence_1/semantic_class/vis_sem_class_371.png  
  inflating: office_2/Sequence_1/semantic_class/semantic_class_372.png  
  inflating: office_2/Sequence_1/semantic_class/vis_sem_class_372.png  
  inflating: office_2/Sequence_1/semantic_class/semantic_class_373.png  
  inflating: office_2/Sequence_1/semantic_class/vis_sem_class_373.png  
  inflating: office_2/Sequence_1/semantic_class/vis_sem_class_376.png  
  inflating: office_2/Sequence_1/semantic_class/semantic_class_377.png  
  inflat

In [ ]:
import glob
import numpy as np
from skimage.io import imread
from torch.utils.data import Dataset
import cv2
import imageio

import os, sys
import logging
import numpy as np
import imageio
import json
import time
import math
import yaml
from collections import defaultdict
from tqdm import trange

import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from imgviz import label_colormap, depth2rgb
from sklearn.metrics import confusion_matrix

In [ ]:
class ReplicaDatasetCache(Dataset):
    def __init__(self, data_dir, train_ids, test_ids, img_h=None, img_w=None):

        traj_file = os.path.join(data_dir, "traj_w_c.txt")
        self.rgb_dir = os.path.join(data_dir, "rgb")
        self.depth_dir = os.path.join(data_dir, "depth")  # depth is in mm uint
        self.semantic_class_dir = os.path.join(data_dir, "semantic_class")
        self.semantic_instance_dir = os.path.join(data_dir, "semantic_instance")
        if not os.path.exists(self.semantic_instance_dir):
            self.semantic_instance_dir = None


        self.train_ids = train_ids
        self.train_num = len(train_ids)
        self.test_ids = test_ids
        self.test_num = len(test_ids)

        self.img_h = img_h
        self.img_w = img_w

        self.Ts_full = np.loadtxt(traj_file, delimiter=" ").reshape(-1, 4, 4)

        self.rgb_list = sorted(glob.glob(self.rgb_dir + '/rgb*.png'), key=lambda file_name: int(file_name.split("_")[-1][:-4]))
        self.depth_list = sorted(glob.glob(self.depth_dir + '/depth*.png'), key=lambda file_name: int(file_name.split("_")[-1][:-4]))
        self.semantic_list = sorted(glob.glob(self.semantic_class_dir + '/semantic_class_*.png'), key=lambda file_name: int(file_name.split("_")[-1][:-4]))
        if self.semantic_instance_dir is not None:
            self.instance_list = sorted(glob.glob(self.semantic_instance_dir + '/semantic_instance_*.png'), key=lambda file_name: int(file_name.split("_")[-1][:-4]))

        self.train_samples = {'image': [], 'depth': [],
                          'semantic': [], 'T_wc': [],
                          'instance': []}

        self.test_samples = {'image': [], 'depth': [],
                          'semantic': [], 'T_wc': [],
                          'instance': []}

       # training samples
        for idx in train_ids:
            image = cv2.imread(self.rgb_list[idx])[:,:,::-1] / 255.0  # change from BGR uinit 8 to RGB float
            depth = cv2.imread(self.depth_list[idx], cv2.IMREAD_UNCHANGED) / 1000.0  # uint16 mm depth, then turn depth from mm to meter
            semantic = cv2.imread(self.semantic_list[idx], cv2.IMREAD_UNCHANGED)
            if self.semantic_instance_dir is not None:
                instance = cv2.imread(self.instance_list[idx], cv2.IMREAD_UNCHANGED) # uint16

            if (self.img_h is not None and self.img_h != image.shape[0]) or \
                    (self.img_w is not None and self.img_w != image.shape[1]):
                image = cv2.resize(image, (self.img_w, self.img_h), interpolation=cv2.INTER_LINEAR)
                depth = cv2.resize(depth, (self.img_w, self.img_h), interpolation=cv2.INTER_LINEAR)
                semantic = cv2.resize(semantic, (self.img_w, self.img_h), interpolation=cv2.INTER_NEAREST)
                if self.semantic_instance_dir is not None:
                    instance = cv2.resize(instance, (self.img_w, self.img_h), interpolation=cv2.INTER_NEAREST)

            T_wc = self.Ts_full[idx]

            self.train_samples["image"].append(image)
            self.train_samples["depth"].append(depth)
            self.train_samples["semantic"].append(semantic)
            if self.semantic_instance_dir is not None:
                self.train_samples["instance"].append(instance)
            self.train_samples["T_wc"].append(T_wc)


        # test samples
        for idx in test_ids:
            image = cv2.imread(self.rgb_list[idx])[:,:,::-1] / 255.0  # change from BGR uinit 8 to RGB float
            depth = cv2.imread(self.depth_list[idx], cv2.IMREAD_UNCHANGED) / 1000.0  # uint16 mm depth, then turn depth from mm to meter
            semantic = cv2.imread(self.semantic_list[idx], cv2.IMREAD_UNCHANGED)
            if self.semantic_instance_dir is not None:
                instance = cv2.imread(self.instance_list[idx], cv2.IMREAD_UNCHANGED) # uint16

            if (self.img_h is not None and self.img_h != image.shape[0]) or \
                    (self.img_w is not None and self.img_w != image.shape[1]):
                image = cv2.resize(image, (self.img_w, self.img_h), interpolation=cv2.INTER_LINEAR)
                depth = cv2.resize(depth, (self.img_w, self.img_h), interpolation=cv2.INTER_LINEAR)
                semantic = cv2.resize(semantic, (self.img_w, self.img_h), interpolation=cv2.INTER_NEAREST)
                if self.semantic_instance_dir is not None:
                    instance = cv2.resize(instance, (self.img_w, self.img_h), interpolation=cv2.INTER_NEAREST)
            T_wc = self.Ts_full[idx]

            self.test_samples["image"].append(image)
            self.test_samples["depth"].append(depth)
            self.test_samples["semantic"].append(semantic)
            if self.semantic_instance_dir is not None:
                self.test_samples["instance"].append(instance)
            self.test_samples["T_wc"].append(T_wc)

        for key in self.test_samples.keys():  # transform list of np array to array with batch dimension
            self.train_samples[key] = np.asarray(self.train_samples[key])
            self.test_samples[key] = np.asarray(self.test_samples[key])

        self.semantic_classes = np.unique(
            np.concatenate(
                (np.unique(self.train_samples["semantic"]),
            np.unique(self.test_samples["semantic"])))).astype(np.uint8)
        print(self.semantic_classes)
        self.num_semantic_class = self.semantic_classes.shape[0]  # number of semantic classes, including the void class of 0

        self.colour_map_np = label_colormap()[self.semantic_classes]
        self.mask_ids = np.ones(self.train_num)  # init self.mask_ids as full ones
        # 1 means the correspinding label map is used for semantic loss during training, while 0 means no semantic loss is applied on this frame

        # remap existing semantic class labels to continuous label ranging from 0 to num_class-1
        self.train_samples["semantic_clean"] = self.train_samples["semantic"].copy()
        self.train_samples["semantic_remap"] = self.train_samples["semantic"].copy()
        self.train_samples["semantic_remap_clean"] = self.train_samples["semantic_clean"].copy()

        self.test_samples["semantic_remap"] = self.test_samples["semantic"].copy()

        for i in range(self.num_semantic_class):
            self.train_samples["semantic_remap"][self.train_samples["semantic"]== self.semantic_classes[i]] = i
            self.train_samples["semantic_remap_clean"][self.train_samples["semantic_clean"]== self.semantic_classes[i]] = i
            self.test_samples["semantic_remap"][self.test_samples["semantic"]== self.semantic_classes[i]] = i


        print()
        print("Training Sample Summary:")
        for key in self.train_samples.keys():
            print("{} has shape of {}, type {}.".format(key, self.train_samples[key].shape, self.train_samples[key].dtype))
        print()
        print("Testing Sample Summary:")
        for key in self.test_samples.keys():
            print("{} has shape of {}, type {}.".format(key, self.test_samples[key].shape, self.test_samples[key].dtype))


    def sample_label_maps(self, sparse_ratio=0.5, K=0, random_sample=False, load_saved=False):
        """
        sparse_ratio means the ratio of removed training images, e.g., 0.3 means 30% of semantic labels are removed
        Input:
            sparse_ratio: the percentage of semantic label frames to be *removed*
            K: the number of frames to be removed, if this is speficied we override the results computed from sparse_ratio
            random_sample: whether to random sample frames or interleavely/evenly sample, True--random sample; False--interleavely sample
            load_saved: use pre-computed mask_ids from previous experiments
        """
        if load_saved is False:
            if K==0:
                K = int(self.train_num*sparse_ratio)  # number of skipped training frames, mask=0

            N = self.train_num-K  # number of used training frames,  mask=1
            assert np.sum(self.mask_ids) == self.train_num  # sanity check that all masks are avaible before sampling

            if K==0: # in case sparse_ratio==0:
                return

            if random_sample:
                self.mask_ids[:K] = 0
                np.random.shuffle(self.mask_ids)
            else:  # sample interleave
                if sparse_ratio<=0.5: # skip less/equal than half frames
                    assert K <= self.train_num/2
                    q, r = divmod(self.train_num, K)
                    indices = [q*i + min(i, r) for i in range(K)]
                    self.mask_ids[indices] = 0

                else: # skip more than half frames
                    assert K > self.train_num/2
                    self.mask_ids = np.zeros_like(self.mask_ids)  # disable all images and  evenly enable N images in total
                    q, r = divmod(self.train_num, N)
                    indices = [q*i + min(i, r) for i in range(N)]
                    self.mask_ids[indices] = 1

            print("{} of {} semantic labels are sampled (sparse ratio: {}).".format(sum(self.mask_ids), len(self.mask_ids), sparse_ratio))
            noisy_sem_dir = os.path.join(self.semantic_class_dir, "noisy_pixel_sems_sr{}".format(sparse_ratio))
            if not os.path.exists(noisy_sem_dir):
                os.makedirs(noisy_sem_dir)
            with open(os.path.join(noisy_sem_dir, "mask_ids.npy"), 'wb') as f:
                np.save(f, self.mask_ids)
        elif load_saved is True:
            noisy_sem_dir = os.path.join(self.semantic_class_dir, "noisy_pixel_sems_sr{}".format(sparse_ratio))
            self.mask_ids = np.load(os.path.join(noisy_sem_dir, "mask_ids.npy"))



    def sample_specific_labels(self, frame_ids, train_ids):
        """
        Only use dense label maps for specific/selected frames.
        """
        assert np.sum(self.mask_ids) == self.train_num  # sanity check that all masks are avaible before sampling

        self.mask_ids = np.zeros_like(self.mask_ids)

        if len(frame_ids)==1 and frame_ids[0] is None:
            # we do not add any semantic supervision
            return

        relative_ids = [train_ids.index(x) for x in frame_ids]

        self.mask_ids[relative_ids] = 1


    def add_pixel_wise_noise_label(self,
        sparse_views=False, sparse_ratio=0.0, random_sample=False,
        noise_ratio=0.0, visualise_save=False, load_saved=False):
        """
        sparse_views: whether we sample a subset of dense semantic labels for training
        sparse_ratio: the ratio of frames to be removed/skipped if sampling a subset of labels
        random_sample: whether to random sample frames or interleavely/evenly sample, True--random sample; False--interleavely sample
        noise_ratio: the ratio of num pixels per-frame to be randomly perturbed
        visualise_save: whether to save the noisy labels into harddrive for later usage
        load_saved: use trained noisy labels for training to ensure consistency betwwen experiments
        """

        if not load_saved:
            if sparse_views:
                self.sample_label_maps(sparse_ratio=sparse_ratio, random_sample=random_sample)
            num_pixel = self.img_h * self.img_w
            num_pixel_noisy = int(num_pixel*noise_ratio)
            train_sem = self.train_samples["semantic_remap"]

            for i in range(len(self.mask_ids)):
                if self.mask_ids[i] == 1:  # add label noise to unmasked/available labels
                    noisy_index_1d = np.random.permutation(num_pixel)[:num_pixel_noisy]
                    faltten_sem = train_sem[i].flatten()

                    faltten_sem[noisy_index_1d] = np.random.choice(self.num_semantic_class, num_pixel_noisy)
                    # we replace the label of randomly selected num_pixel_noisy pixels to random labels from [1, self.num_semantic_class], 0 class is the none class
                    train_sem[i] = faltten_sem.reshape(self.img_h, self.img_w)

            print("{} of {} semantic labels are added noise {} percent area ratio.".format(sum(self.mask_ids), len(self.mask_ids), noise_ratio))

            if visualise_save:
                noisy_sem_dir = os.path.join(self.semantic_class_dir, "noisy_pixel_sems_sr{}_nr{}".format(sparse_ratio, noise_ratio))
                if not os.path.exists(noisy_sem_dir):
                    os.makedirs(noisy_sem_dir)
                with open(os.path.join(noisy_sem_dir, "mask_ids.npy"), 'wb') as f:
                    np.save(f, self.mask_ids)

                vis_noisy_semantic_list = []
                vis_semantic_clean_list = []

                colour_map_np = self.colour_map_np
                # 101 classes in total from Replica, select the existing class from total colour map

                semantic_remap = self.train_samples["semantic_remap"] # [H, W, 3]
                semantic_remap_clean = self.train_samples["semantic_remap_clean"] # [H, W, 3]

                # save semantic labels
                for i in range(len(self.mask_ids)):
                    if self.mask_ids[i] == 1:
                        vis_noisy_semantic = colour_map_np[semantic_remap[i]] # [H, W, 3]
                        vis_semantic_clean = colour_map_np[semantic_remap_clean[i]] # [H, W, 3]

                        imageio.imwrite(os.path.join(noisy_sem_dir, "semantic_class_{}.png".format(i)), semantic_remap[i])
                        imageio.imwrite(os.path.join(noisy_sem_dir, "vis_sem_class_{}.png".format(i)), vis_noisy_semantic)

                        vis_noisy_semantic_list.append(vis_noisy_semantic)
                        vis_semantic_clean_list.append(vis_semantic_clean)
                    else:
                        # for mask_ids of 0, we skip these frames during training and do not add noise
                        vis_noisy_semantic = colour_map_np[semantic_remap[i]] # [H, W, 3]
                        vis_semantic_clean = colour_map_np[semantic_remap_clean[i]] # [H, W, 3]
                        assert np.all(vis_noisy_semantic==vis_semantic_clean) # apply this check to skipped frames

                        imageio.imwrite(os.path.join(noisy_sem_dir, "semantic_class_{}.png".format(i)), semantic_remap[i])
                        imageio.imwrite(os.path.join(noisy_sem_dir, "vis_sem_class_{}.png".format(i)), vis_noisy_semantic)

                        vis_noisy_semantic_list.append(vis_noisy_semantic)
                        vis_semantic_clean_list.append(vis_semantic_clean)

                imageio.mimwrite(os.path.join(noisy_sem_dir, 'noisy_sem_ratio_{}.mp4'.format(noise_ratio)),
                        np.stack(vis_noisy_semantic_list, 0), fps=30, quality=8)

                imageio.mimwrite(os.path.join(noisy_sem_dir, 'clean_sem.mp4'),
                        np.stack(vis_semantic_clean_list, 0), fps=30, quality=8)
        else:
            print("Load saved noisy labels.")
            noisy_sem_dir = os.path.join(self.semantic_class_dir, "noisy_pixel_sems_sr{}_nr{}".format(sparse_ratio, noise_ratio))
            assert os.path.exists(noisy_sem_dir)
            self.mask_ids = np.load(os.path.join(noisy_sem_dir, "mask_ids.npy"))
            semantic_img_list = []
            semantic_path_list = sorted(glob.glob(noisy_sem_dir + '/semantic_class_*.png'), key=lambda file_name: int(file_name.split("_")[-1][:-4]))
            assert len(semantic_path_list)>0
            for idx in range(len(self.mask_ids)):
                semantic = imread(semantic_path_list[idx])
                semantic_img_list.append(semantic)
            self.train_samples["semantic_remap"]  = np.asarray(semantic_img_list)


    def add_instance_wise_noise_label(self, sparse_views=False, sparse_ratio=0.0, random_sample=False,
            flip_ratio=0.0, uniform_flip=False,
            instance_id=[3, 6, 7, 9, 11, 12, 13, 48],
            load_saved=False,
            visualise_save=False):

        """ In this function, we try to test if semantic-NERF can correct the wrong instance label after fusion (training).
        For selected instances, we randomly pick a portion of frames and change their class labels.
        Input:
            sparse_views: if we use a subset of sampled original training set or not.
            sparse_ratio: the ratio of frames to be dropped.
            random_sample: whether to random sample frames or interleavely/evenly sample, True--random sample; False--interleavely sample
            flip_ratio: for all the frames containing certain instances, the ratio of changing labels
            uniform_flip: True: after sorting the candidate frames by instance area ratio,
                                  we uniform sample frames to flip certain instances' semantic class;
                          False: we take the frames with least instance area ratio to change color.
            instance_id: the instance id of all 8 chairs in Replica Room_2, used for adding region-wise noise
            load_saved: whether to load the saved self.mask_ids or not
            visualise_save: If true, save processed partial labels into local harddrive/folders for futher usage.


        """
        num_pixel = self.img_w * self.img_h

        if not load_saved:
            if sparse_views:
                self.sample_label_maps(sparse_ratio=sparse_ratio, random_sample=random_sample, load_saved=load_saved)
            assert self.semantic_instance_dir is not None
            # instance_id = [3, 6, 7, 9, 11,12, 13, 48]
            # instance_maps_dict = dict.fromkeys(instance_id, [])  # using this one will make all the keys share the same value due to list [] is mutable
            instance_maps_dict = dict.fromkeys(instance_id)
            for k in instance_maps_dict.keys():
                instance_maps_dict[k] = list()


            # find which training images contrain the instance we want to flip labels
            for img_idx in range(self.train_num):
                instance_label_map = self.train_samples["instance"][img_idx]
                for ins_idx in instance_id:
                    instance_ratio = np.sum(instance_label_map==ins_idx)/num_pixel
                    if instance_ratio > 0 and self.mask_ids[img_idx]==1: # larger than 1% image area and the image is also sampled into training set
                        instance_maps_dict[ins_idx].append([img_idx, instance_ratio])

            num_frame_per_instance_id = np.asarray([len(x) for x in  instance_maps_dict.values()])
            num_flip_frame_per_instance_id =  (num_frame_per_instance_id*flip_ratio).astype(np.int)

            for k, v in instance_maps_dict.items():
                instance_maps_dict[k] = sorted(instance_maps_dict[k], key=lambda x: x[1])  # sorted, default is ascending order
            if not uniform_flip:
            # we flip the labels with minimum area ratio,
            # the intuition is that the observation is partial and is likely to be wrong.
                for i in range(len(instance_id)):  # loop over instance id
                    selected_frame_id = [x[0] for x in instance_maps_dict[instance_id[i]][:num_flip_frame_per_instance_id[i]]]
                    for m in selected_frame_id: # loop over image ids having the selected instance
                        self.train_samples["semantic_remap"][m][self.train_samples["instance"][m]==instance_id[i]] = np.random.choice(self.num_semantic_class, 1)
            else:
                if flip_ratio<=0.5: # flip less/equal than half frames
                    for i in range(len(instance_id)):  # loop over instance id
                        K = num_flip_frame_per_instance_id[i]
                        q, r = divmod(num_frame_per_instance_id[i], K)
                        indices_to_flip = [q*i + min(i, r) for i in range(K)]
                        valid_frame_id_list = [x[0] for x in instance_maps_dict[instance_id[i]]]
                        selected_frame_id = [valid_frame_id_list[flip_id] for flip_id in indices_to_flip]
                        for m in selected_frame_id: # loop over image ids having the selected instance
                            self.train_samples["semantic_remap"][m][self.train_samples["instance"][m]==instance_id[i]] = np.random.choice(self.num_semantic_class, 1)

                else: # flip more than half frames
                    for i in range(len(instance_id)):  # loop over instance id
                        K = num_flip_frame_per_instance_id[i]
                        N = num_frame_per_instance_id[i] - K
                        q, r = divmod(num_frame_per_instance_id[i], N)
                        indices_NOT_flip = [q*i + min(i, r) for i in range(N)]
                        indices_to_flip = [x for x in range(num_frame_per_instance_id[i]) if x not in indices_NOT_flip]
                        valid_frame_id_list = [x[0] for x in instance_maps_dict[instance_id[i]]]
                        selected_frame_id = [valid_frame_id_list[flip_id] for flip_id in indices_to_flip]
                        for m in selected_frame_id: # loop over image ids having the selected instance
                            self.train_samples["semantic_remap"][m][self.train_samples["instance"][m]==instance_id[i]] = np.random.choice(self.num_semantic_class, 1)

            colour_map_np = self.colour_map_np
            vis_flip_semantic = [colour_map_np[sem] for sem in self.train_samples["semantic_remap"]]
            vis_gt_semantic = [colour_map_np[sem] for sem in self.train_samples["semantic_remap_clean"]]

            if visualise_save:
                flip_sem_dir  = os.path.join(self.semantic_class_dir, "flipped_chair_nr_{}".format(flip_ratio))
                if not os.path.exists(flip_sem_dir):
                    os.makedirs(flip_sem_dir)

                with open(os.path.join(flip_sem_dir, "mask_ids.npy"), 'wb') as f:
                    np.save(f, self.mask_ids)

                for i in range(len(vis_flip_semantic)):
                    imageio.imwrite(os.path.join(flip_sem_dir, "semantic_class_{}.png".format(i)), self.train_samples["semantic_remap"][i])
                    imageio.imwrite(os.path.join(flip_sem_dir, "vis_sem_class_{}.png".format(i)), vis_flip_semantic[i])
                    imageio.imwrite(os.path.join(flip_sem_dir, "vis_gt_{}.png".format(i)), vis_gt_semantic[i])
        else:
            print("Load saved noisy labels.")
            flip_sem_dir  = os.path.join(self.semantic_class_dir, "flipped_chair_nr_{}".format(flip_ratio))
            assert os.path.exists(flip_sem_dir)
            self.mask_ids = np.load(os.path.join(flip_sem_dir, "mask_ids.npy"))
            semantic_img_list = []
            semantic_path_list = sorted(glob.glob(flip_sem_dir + '/semantic_class_*.png'), key=lambda file_name: int(file_name.split("_")[-1][:-4]))
            assert len(semantic_path_list)>0
            for idx in range(len(self.mask_ids)):
                semantic = imread(semantic_path_list[idx])
                semantic_img_list.append(semantic)
            self.train_samples["semantic_remap"]  = np.asarray(semantic_img_list)

    def super_resolve_label(self, down_scale_factor=8, dense_supervision=True):
        """ In super-resolution mode, to create training supervisions, we downscale the ground truth label by certain scaling factor to
        throw away information. We then upscale the image back to original size.

        Two setups for upscaling:
            (1) Sparse label: we set the interpolated label pixels to void label==0, so we only have losses on grid of every 8 pixels
            (2) Dense label: we penalise also on interpolated pixel values

        down_scale_factor: the scaling factor for down-sampling and up-sampling
        dense_supervision: dense label mode or not.
        """
        if down_scale_factor==1:
            return
        if dense_supervision:  # for dense labelling,  we down-scale and up-scale label maps again
            scaled_low_res_train_label = []
            for i in range(self.train_num):
                low_res_label = cv2.resize(self.train_samples["semantic_remap"][i],
                (self.img_w//down_scale_factor, self.img_h//down_scale_factor),
                interpolation=cv2.INTER_NEAREST)

                scaled_low_res_label = cv2.resize(low_res_label, (self.img_w, self.img_h), interpolation=cv2.INTER_NEAREST)
                scaled_low_res_train_label.append(scaled_low_res_label)

            scaled_low_res_train_label = np.asarray(scaled_low_res_train_label)

            self.train_samples["semantic_remap"] = scaled_low_res_train_label

        else: # for sparse labelling, we only penalise strictly on valid pixel positions
            valid_low_res_pixel_mask = np.zeros((self.img_h, self.img_w))
            valid_low_res_pixel_mask[::down_scale_factor, ::down_scale_factor]=1
            self.train_samples["semantic_remap"] = (self.train_samples["semantic_remap"]*valid_low_res_pixel_mask[None,...]).astype(np.uint8)
            # we mask all the decimated pixel label to void class==0

    def simulate_user_click_partial(self, perc=0, load_saved=False, visualise_save=True):
        """
        Generate partial label maps for label propagation task.
        perc: the percentage of pixels per class per image to be preserved to simulate partial user clicks
            0: single-clicks
            1: 1% user clicks
            5: 5% user clicks

        load_saved: If true, load saved partial clicks to guarantee reproductability. False, create new partial laels
        visualise_save: If true, save processed partial labels into local harddrive/folders for futher usage like visualisation.
        """
        assert perc<=100 and perc >= 0
        assert self.train_num == self.train_samples["semantic_remap"].shape[0]
        single_click=True if perc==0 else False # single_click: whether to use single click only from each class
        perc = perc/100.0 # make perc value into percentage
        if not load_saved:

            if single_click:
                click_semantic_map = []
                for i in range(self.train_num):
                    if (i+1)%10==0:
                        print("Generating partial label of ratio {} for frame {}/{}.".format(perc, i, self.train_num))
                    im  = self.train_samples["semantic_remap"][i]
                    void_class = [0]
                    label_class = np.unique(im).tolist()
                    valid_class = [i for i in label_class if i not in void_class]
                    im_ = np.zeros_like(im)
                    for l in valid_class:
                        label_idx = np.transpose(np.nonzero(im == l))
                        sample_ind = np.random.choice(label_idx.shape[0], 1, replace=False)
                        label_idx_ = label_idx[sample_ind]
                        im_[label_idx_[:, 0], label_idx_[:, 1]] = l
                    click_semantic_map.append(im_)
                click_semantic_map = np.asarray(click_semantic_map).astype(np.uint8)
                self.train_samples["semantic_remap"] = click_semantic_map

                print('Partial Label images with centroid sampling (extreme) has completed.')

            elif perc>0 and not single_click:
                click_semantic_map = []
                for i in range(self.train_num):
                    if (i+1)%10==0:
                        print("Generating partial label of ratio {} for frame {}/{}.".format(perc, i, self.train_num))
                    im  = self.train_samples["semantic_remap"][i]
                    void_class = [0]
                    label_class = np.unique(im).tolist() # find the unique class-ids in the current training label
                    valid_class = [c for c in label_class if c not in void_class]

                    im_ = np.zeros_like(im)
                    for l in valid_class:
                        label_mask = np.zeros_like(im)
                        label_mask_ = im == l # binary mask of pixels equal to class-l
                        label_idx = np.transpose(np.nonzero(label_mask_)) # Nx2
                        sample_ind = np.random.choice(label_idx.shape[0], 1, replace=False) # shape [1,]
                        label_idx_ = label_idx[sample_ind] # shape [1, 2]
                        target_num = int(perc * label_mask_.sum()) # find the target and total number of pixels belong to class-l in the current image
                        label_mask[label_idx_[0, 0], label_idx_[0, 1]] = 1 # full-zero mask with only selected pixel to be 1
                        label_mask_true = label_mask
                        # label_mask_true initially has only 1 True pixel, we continuously grow mask until reach expected percentage

                        while label_mask_true.sum() < target_num:
                            num_before_grow = label_mask_true.sum()
                            label_mask = cv2.dilate(label_mask, kernel=np.ones([5, 5]))
                            label_mask_true = label_mask * label_mask_
                            num_after_grow = label_mask_true.sum()
                            if num_after_grow==num_before_grow:
                                print("Initialise Another Seed for Growing!")
                                # The current region stop growing which means the very local area has been filled,
                                #  so we need to initiate another seed to keep it growing
                                uncovered_region_mask = label_mask_ - label_mask_true # pixels which are equal to 1 are un-sampled regions and belong to current class
                                label_idx = np.transpose(np.nonzero(uncovered_region_mask)) # Nx2
                                sample_ind = np.random.choice(label_idx.shape[0], 1, replace=False) # shape [1,]
                                label_idx_ = label_idx[sample_ind] # shape [1, 2]
                                label_mask[label_idx_[0, 0], label_idx_[0, 1]] = 1

                        im_[label_mask_true.astype(bool)] = l
                    click_semantic_map.append(im_)

                click_semantic_map = np.asarray(click_semantic_map).astype(np.uint8)
                self.train_samples["semantic_remap"] = click_semantic_map
                print('Partial Label images with centroid sampling has completed.')
            else:
                assert False

            if visualise_save:
                partial_sem_dir = os.path.join(self.semantic_class_dir, "partial_perc_{}".format(perc))
                if not os.path.exists(partial_sem_dir):
                    os.makedirs(partial_sem_dir)
                colour_map_np = self.colour_map_np
                vis_partial_sem = []
                for i in range(self.train_num):
                    vis_partial_semantic = colour_map_np[self.train_samples["semantic_remap"][i]] # [H, W, 3]
                    imageio.imwrite(os.path.join(partial_sem_dir, "semantic_class_{}.png".format(i)), self.train_samples["semantic_remap"][i])
                    imageio.imwrite(os.path.join(partial_sem_dir, "vis_sem_class_{}.png".format(i)), vis_partial_semantic)
                    vis_partial_sem.append(vis_partial_semantic)

                imageio.mimwrite(os.path.join(partial_sem_dir, 'partial_sem.mp4'), self.train_samples["semantic_remap"], fps=30, quality=8)
                imageio.mimwrite(os.path.join(partial_sem_dir, 'vis_partial_sem.mp4'), np.stack(vis_partial_sem, 0), fps=30, quality=8)

        else: # load saved single-click/partial semantics
            saved_partial_sem_dir = os.path.join(self.semantic_class_dir, "partial_perc_{}".format(perc))
            semantic_img_list = []
            semantic_path_list = sorted(glob.glob(saved_partial_sem_dir + '/semantic_class_*.png'), key=lambda file_name: int(file_name.split("_")[-1][:-4]))
            assert len(semantic_path_list)>0
            for idx in range(self.train_num):
                semantic = imread(semantic_path_list[idx])
                semantic_img_list.append(semantic)
            self.train_samples["semantic_remap"]  = np.asarray(semantic_img_list).astype(np.uint8)

In [ ]:
total_num = 900
step = 5
train_ids = list(range(0, total_num, step))
test_ids = [x+step//2 for x in train_ids]
width = 320
height = 240

In [ ]:
dt = ReplicaDatasetCache("/content/office_2/Sequence_1", train_ids, test_ids, img_h = height, img_w = width)

[ 0 10 14 17 20 22 29 31 37 40 47 56 76 78 80 82 83 87 92 93 95 97]

Training Sample Summary:
image has shape of (180, 240, 320, 3), type float64.
depth has shape of (180, 240, 320), type float64.
semantic has shape of (180, 240, 320), type uint8.
T_wc has shape of (180, 4, 4), type float64.
instance has shape of (0,), type float64.
semantic_clean has shape of (180, 240, 320), type uint8.
semantic_remap has shape of (180, 240, 320), type uint8.
semantic_remap_clean has shape of (180, 240, 320), type uint8.

Testing Sample Summary:
image has shape of (180, 240, 320, 3), type float64.
depth has shape of (180, 240, 320), type float64.
semantic has shape of (180, 240, 320), type uint8.
T_wc has shape of (180, 4, 4), type float64.
instance has shape of (0,), type float64.
semantic_remap has shape of (180, 240, 320), type uint8.


In [ ]:
def sample_stratified(
  rays_o: torch.Tensor,
  rays_d: torch.Tensor,
  near: float,
  far: float,
  n_samples: int,
  perturb: bool = True,
  inverse_depth: bool = False
):
  r"""
  Sample along ray from regularly-spaced bins.
  """

  # Grab samples for space integration along ray
  t_vals = torch.linspace(0., 1., n_samples, device=rays_o.device)
  if not inverse_depth:
    # Sample linearly between `near` and `far`
    z_vals = near * (1.-t_vals) + far * (t_vals)
  else:
    # Sample linearly in inverse depth (disparity)
    z_vals = 1./(1./near * (1.-t_vals) + 1./far * (t_vals))

  # Draw uniform samples from bins along ray
  if perturb:
    mids = .5 * (z_vals[1:] + z_vals[:-1])
    upper = torch.concat([mids, z_vals[-1:]], dim=-1)
    lower = torch.concat([z_vals[:1], mids], dim=-1)
    t_rand = torch.rand([n_samples], device=z_vals.device)
    z_vals = lower + (upper - lower) * t_rand
  z_vals = z_vals.expand(list(rays_o.shape[:-1]) + [n_samples])

  # Apply scale from `rays_d` and offset from `rays_o` to samples
  # pts: (width, height, n_samples, 3)
  pts = rays_o[..., None, :] + rays_d[..., None, :] * z_vals[..., :, None]
  return pts, z_vals

def get_rays_camera(B, H, W, fx, fy,  cx, cy, depth_type, convention="opencv"):

    assert depth_type == "z" or depth_type == "euclidean"
    i, j = torch.meshgrid(torch.arange(W), torch.arange(H))  # pytorch's meshgrid has indexing='ij', we transpose to "xy" moode

    i = i.t().float()
    j = j.t().float()

    size = [B, H, W]

    i_batch = torch.empty(size)
    j_batch = torch.empty(size)
    i_batch[:, :, :] = i[None, :, :]
    j_batch[:, :, :] = j[None, :, :]

    if convention == "opencv":
        x = (i_batch - cx) / fx
        y = (j_batch - cy) / fy
        z = torch.ones(size)
    elif convention == "opengl":
        x = (i_batch - cx) / fx
        y = -(j_batch - cy) / fy
        z = -torch.ones(size)
    else:
        assert False

    dirs = torch.stack((x, y, z), dim=3)  # shape of [B, H, W, 3]

    if depth_type == 'euclidean':
        norm = torch.norm(dirs, dim=3, keepdim=True)
        dirs = dirs * (1. / norm)

    return dirs


def get_rays_world(T_WC, dirs_C):
    R_WC = T_WC[:, :3, :3]  # Bx3x3
    dirs_W = torch.matmul(R_WC[:, None, ...], dirs_C[..., None]).squeeze(-1)
    origins = T_WC[:, :3, -1]  # Bx3
    origins = torch.broadcast_tensors(origins[:, None, :], dirs_W)[0]
    return origins, dirs_W


def create_rays(num_rays, Ts_c2w, height, width, fx, fy, cx, cy, near, far, c2w_staticcam=None, depth_type="z",
              use_viewdirs=True, convention="opencv"):
    """
    convention:
    "opencv" or "opengl". It defines the coordinates convention of rays from cameras.
    OpenCv defines x,y,z as right, down, forward while OpenGl defines x,y,z as right, up, backward (camera looking towards forward direction still, -z!)
    Note: Use either convention is fine, but the corresponding pose should follow the same convention.

    """
    print('prepare rays')

    rays_cam = get_rays_camera(num_rays, height, width, fx, fy, cx, cy, depth_type=depth_type, convention=convention) # [N, H, W, 3]

    dirs_C = rays_cam.view(num_rays, -1, 3)  # [N, HW, 3]
    rays_o, rays_d = get_rays_world(Ts_c2w, dirs_C)  # origins: [B, HW, 3], dirs_W: [B, HW, 3]

    if use_viewdirs:
        # provide ray directions as input
        viewdirs = rays_d
        if c2w_staticcam is not None:
            # c2w_staticcam: If not None, use this transformation matrix for camera,
            # while using other c2w argument for viewing directions.
            # special case to visualize effect of viewdirs
            rays_o, rays_d = get_rays_world(c2w_staticcam, dirs_C)  # origins: [B, HW, 3], dirs_W: [B, HW, 3]

        viewdirs = viewdirs / torch.norm(viewdirs, dim=-1, keepdim=True).float()

    near, far = near * torch.ones_like(rays_d[..., :1]), far * torch.ones_like(rays_d[..., :1])
    rays = torch.cat([rays_o, rays_d, near, far], -1)

    if use_viewdirs:
        rays = torch.cat([rays, viewdirs], -1)
    return rays

def run_network(inputs, viewdirs, fn, embed_fn, embeddirs_fn, netchunk=1024 * 64):
    """Prepares inputs and applies network 'fn'.

    Input: [N_rays, N_samples, 3]
    """
    inputs_flat = torch.reshape(inputs, [-1, inputs.shape[-1]])
    embedded = embed_fn(inputs_flat)

    if viewdirs is not None:
        input_dirs = viewdirs[:, None].expand(inputs.shape)
        input_dirs_flat = torch.reshape(input_dirs, [-1, input_dirs.shape[-1]])
        embedded_dirs = embeddirs_fn(input_dirs_flat)
        embedded = torch.cat([embedded, embedded_dirs], -1)


    #print(embedded.shape)
    outputs_flat = torch.cat([fn(embedded[i:i + netchunk]) for i in range(0, embedded.shape[0], netchunk)], 0)
    outputs = torch.reshape(outputs_flat, list(inputs.shape[:-1]) + [outputs_flat.shape[-1]])
    return outputs


def raw2outputs(raw, z_vals, rays_d, raw_noise_std=0, white_bkgd=False, enable_semantic=True,
                num_sem_class=0, endpoint_feat=False):
    """Transforms model's predictions to semantically meaningful values.
    Args:
        raw: [num_rays, num_samples along ray, 4]. Prediction from model.
        z_vals: [num_rays, num_samples along ray]. Integration time.
        rays_d: [num_rays, 3]. Direction of each ray.
        raw_noise_std: random perturbations added to ray samples

    Returns:
        rgb_map: [num_rays, 3]. Estimated RGB color of a ray.
        disp_map: [num_rays]. Disparity map. Inverse of depth map.
        acc_map: [num_rays]. Sum of weights along each ray.
        weights: [num_rays, num_samples]. Weights assigned to each sampled color.
        depth_map: [num_rays]. Estimated distance to object.
    """
    raw2alpha = lambda raw, dists, act_fn=F.relu: 1.-torch.exp(-act_fn(raw)*dists)

    dists = z_vals[..., 1:] - z_vals[..., :-1]  # # (N_rays, N_samples_-1)
    dists = torch.cat([dists, torch.Tensor([1e10]).expand(dists[..., :1].shape).cuda()], -1)  # [N_rays, N_samples]

    # Multiply each distance by the norm of its corresponding direction ray
    # to convert to real world distance (accounts for non-unit directions).
    dists = dists * torch.norm(rays_d[..., None, :], dim=-1)

    rgb = torch.sigmoid(raw[..., :3])  # [N_rays, N_samples, 3]

    if raw_noise_std > 0.:
        noise = torch.randn(raw[..., 3].shape) * raw_noise_std
        noise = noise.cuda()
    else:
        noise = 0.

    alpha = raw2alpha(raw[..., 3] + noise, dists)  # [N_rays, N_samples]


    # weights = alpha * tf.math.cumprod(1.-alpha + 1e-10, -1, exclusive=True)
    weights = alpha * torch.cumprod(torch.cat([torch.ones((alpha.shape[0], 1)).cuda(), 1.-alpha + 1e-10], -1), -1)[:, :-1]
    # [1, 1-a1, 1-a2, ...]
    # [N_rays, N_samples+1] sliced by [:, :-1] to [N_rays, N_samples]

    rgb_map = torch.sum(weights[..., None] * rgb, -2)  # [N_rays, 3]
    # [N_rays, 3], the accumulated opacity along the rays, equals "1 - (1-a1)(1-a2)...(1-an)" mathematically

    if enable_semantic:
        assert num_sem_class>0
        # https://discuss.pytorch.org/t/multi-class-cross-entropy-loss-and-softmax-in-pytorch/24920/2
        sem_logits = raw[..., 4:4+num_sem_class]  # [N_rays, N_samples, num_class]
        sem_map = torch.sum(weights[..., None] * sem_logits, -2)  # [N_rays, num_class]
    else:
        sem_map = torch.tensor(0)


    if endpoint_feat:
        feat = raw[..., -128:] # [N_rays, N_samples, feat_dim] take the last 128 dim from predictions
        feat_map = torch.sum(weights[..., None] * feat, -2)  # [N_rays, feat_dim]
    else:
        feat_map = torch.tensor(0)

    depth_map = torch.sum(weights * z_vals, -1)  # (N_rays,)
    disp_map = 1./torch.max(1e-10 * torch.ones_like(depth_map), depth_map / torch.sum(weights, -1))
    acc_map = torch.sum(weights, -1)

    if white_bkgd:
        rgb_map = rgb_map + (1.-acc_map[..., None])
        if enable_semantic:
            sem_map = sem_map + (1.-acc_map[..., None])

    return rgb_map, disp_map, acc_map, weights, depth_map, sem_map, feat_map


def sample_pdf(bins, weights, N_samples, det=False):
    """ Sample @N_importance samples from @bins with distribution defined by @weights.

    Inputs:
        bins: N_rays x (N_samples_coarse - 1)
        weights: N_rays x (N_samples_coarse - 2)
        N_samples: N_samples_fine
        det: deterministic or not
    """
    # Get pdf
    weights = weights + 1e-5  # prevent nans, prevent division by zero (don't do inplace op!)
    pdf = weights / torch.sum(weights, -1, keepdim=True)
    cdf = torch.cumsum(pdf, -1)  # N_rays x (N_samples - 2)
    cdf = torch.cat([torch.zeros_like(cdf[..., :1]), cdf], -1)  # N_rays x (N_samples_coarse - 1)
    # padded to 0~1 inclusive, (N_rays, N_samples-1)

    # Take uniform samples
    if det:  # generate deterministic samples
        u = torch.linspace(0., 1., steps=N_samples,  device=bins.device)
        u = u.expand(list(cdf.shape[:-1]) + [N_samples])
    else:
        u = torch.rand(list(cdf.shape[:-1]) + [N_samples],  device=bins.device)
        # (N_rays, N_samples_fine)

    # Invert CDF
    u = u.contiguous()
    inds = torch.searchsorted(cdf.detach(), u, right=True)  # N_rays x N_samples_fine
    below = torch.max(torch.zeros_like(inds-1), inds-1)
    above = torch.min((cdf.shape[-1]-1) * torch.ones_like(inds), inds)
    inds_g = torch.stack([below, above], -1)  # (N_rays, N_samples_fine, 2)

    matched_shape = [inds_g.shape[0], inds_g.shape[1], cdf.shape[-1]]  # (N_rays, N_samples_fine, N_samples_coarse - 1)

    cdf_g = torch.gather(cdf.unsqueeze(1).expand(matched_shape), 2, inds_g)  # N_rays, N_samples_fine, 2
    bins_g = torch.gather(bins.unsqueeze(1).expand(matched_shape), 2, inds_g)  # N_rays, N_samples_fine, 2

    denom = (cdf_g[..., 1]-cdf_g[..., 0])  # # N_rays, N_samples_fine
    denom = torch.where(denom < 1e-5, torch.ones_like(denom), denom)
    # denom equals 0 means a bin has weight 0, in which case it will not be sampled
    # anyway, therefore any value for it is fine (set to 1 here)

    t = (u-cdf_g[..., 0])/denom
    samples = bins_g[..., 0] + t * (bins_g[...,1]-bins_g[...,0])

    return samples

def batchify_rays(render_fn, rays_flat, chunk=1024 * 32):
    """Render rays in smaller minibatches to avoid OOM.
    """
    all_ret = {}
    for i in range(0, rays_flat.shape[0], chunk):
        ret = render_fn(rays_flat[i:i + chunk])
        for k in ret:
            if k not in all_ret:
                all_ret[k] = []
            all_ret[k].append(ret[k])

    all_ret = {k: torch.cat(all_ret[k], 0) for k in all_ret}
    return all_ret

def nanmean(data, **args):
    # This makes it ignore the first 'background' class
    return np.ma.masked_array(data, np.isnan(data)).mean(**args)
    # In np.ma.masked_array(data, np.isnan(data), elements of data == np.nan is invalid and will be ingorned during computation of np.mean()

def calculate_segmentation_metrics(true_labels, predicted_labels, number_classes, ignore_label):
    if (true_labels == ignore_label).all():
        return [0]*4

    true_labels = true_labels.flatten()
    predicted_labels = predicted_labels.flatten()
    valid_pix_ids = true_labels!=ignore_label
    predicted_labels = predicted_labels[valid_pix_ids]
    true_labels = true_labels[valid_pix_ids]

    conf_mat = confusion_matrix(true_labels, predicted_labels, labels=list(range(number_classes)))
    norm_conf_mat = np.transpose(
        np.transpose(conf_mat) / conf_mat.astype(np.float).sum(axis=1))

    missing_class_mask = np.isnan(norm_conf_mat.sum(1)) # missing class will have NaN at corresponding class
    exsiting_class_mask = ~ missing_class_mask

    class_average_accuracy = nanmean(np.diagonal(norm_conf_mat))
    total_accuracy = (np.sum(np.diagonal(conf_mat)) / np.sum(conf_mat))
    ious = np.zeros(number_classes)
    for class_id in range(number_classes):
        ious[class_id] = (conf_mat[class_id, class_id] / (
                np.sum(conf_mat[class_id, :]) + np.sum(conf_mat[:, class_id]) -
                conf_mat[class_id, class_id]))
    miou = nanmean(ious)
    miou_valid_class = np.mean(ious[exsiting_class_mask])
    return miou, miou_valid_class, total_accuracy, class_average_accuracy, ious


In [ ]:
class PositionalEncoder(nn.Module):
  r"""
  Sine-cosine positional encoder for input points.
  """
  def __init__(
    self,
    d_input: int,
    n_freqs: int,
    log_space: bool = False
  ):
    super().__init__()
    self.d_input = d_input
    self.n_freqs = n_freqs
    self.log_space = log_space
    self.d_output = d_input * (1 + 2 * self.n_freqs)
    self.embed_fns = [lambda x: x]

    # Define frequencies in either linear or log scale
    if self.log_space:
      freq_bands = 2.**torch.linspace(0., self.n_freqs - 1, self.n_freqs)
    else:
      freq_bands = torch.linspace(2.**0., 2.**(self.n_freqs - 1), self.n_freqs)

    # Alternate sin and cos
    for freq in freq_bands:
      self.embed_fns.append(lambda x, freq=freq: torch.sin(x * freq))
      self.embed_fns.append(lambda x, freq=freq: torch.cos(x * freq))

  def forward(
    self,
    x
  ) -> torch.Tensor:
    r"""
    Apply positional encoding to input.
    """
    return torch.concat([fn(x) for fn in self.embed_fns], dim=-1)


In [ ]:
class Semantic_NeRF(nn.Module):
    """
    Compared to the NeRF class wich also predicts semantic logits from MLPs, here we make the semantic label only a function of 3D position
    instead of both positon and viewing directions.
    """
    def __init__(self, enable_semantic, num_semantic_classes, D=8, W=256, input_ch=3, input_ch_views=3, output_ch=4, skips=[4], use_viewdirs=False,
                 ):
        super(Semantic_NeRF, self).__init__()
        """
                D: number of layers for density (sigma) encoder
                W: number of hidden units in each layer
                input_ch: number of input channels for xyz (3+3*10*2=63 by default)
                in_channels_dir: number of input channels for direction (3+3*4*2=27 by default)
                skips: layer index to add skip connection in the Dth layer
        """
        self.D = D
        self.W = W
        self.input_ch = input_ch
        self.input_ch_views = input_ch_views
        self.skips = skips
        self.use_viewdirs = use_viewdirs
        self.enable_semantic = enable_semantic

        # build the encoder
        self.pts_linears = nn.ModuleList(
            [nn.Linear(input_ch, W)] + [nn.Linear(W, W) if i not in self.skips else nn.Linear(W + input_ch, W) for i in
                                        range(D - 1)])

        ### Implementation according to the official code release (https://github.com/bmild/nerf/blob/master/run_nerf_helpers.py#L104-L105)

        # Another layer is used to
        self.views_linears = nn.ModuleList([nn.Linear(input_ch_views + W, W // 2)])
        if use_viewdirs:
            self.feature_linear = nn.Linear(W, W)
            self.alpha_linear = nn.Linear(W, 1)
            if enable_semantic:
                self.semantic_linear = nn.Sequential(nn.Linear(W, W//2),nn.ReLU(W // 2), nn.Linear(W // 2, num_semantic_classes))
            self.rgb_linear = nn.Linear(W // 2, 3)
        else:
            self.output_linear = nn.Linear(W, output_ch)

    def forward(self, x, show_endpoint=False):
        """
        Encodes input (xyz+dir) to rgb+sigma+semantics raw output
        Inputs:
            x: (B, self.in_channels_xyz(+self.in_channels_dir))
               the embedded vector of 3D xyz position and viewing direction
        """
        input_pts, input_views = torch.split(x, [self.input_ch, self.input_ch_views], dim=-1)
        h = input_pts
        for i, l in enumerate(self.pts_linears):
            h = self.pts_linears[i](h)
            h = F.relu(h)
            if i in self.skips:
                h = torch.cat([input_pts, h], -1)

        if self.use_viewdirs:
            # if using view-dirs, output occupancy alpha as well as features for concatenation
            alpha = self.alpha_linear(h)
            if self.enable_semantic:
                sem_logits = self.semantic_linear(h)
            feature = self.feature_linear(h)

            h = torch.cat([feature, input_views], -1)

            for i, l in enumerate(self.views_linears):
                h = self.views_linears[i](h)
                h = F.relu(h)

            if show_endpoint:
                endpoint_feat = h
            rgb = self.rgb_linear(h)

            if self.enable_semantic:
                outputs = torch.cat([rgb, alpha, sem_logits], -1)
            else:
                outputs = torch.cat([rgb, alpha], -1)
        else:
            outputs = self.output_linear(h)

        if show_endpoint is False:
            return outputs
        else:
            return torch.cat([outputs, endpoint_feat], -1)

In [ ]:
class SSRTrainer(object):
    def __init__(self, config):
        super(SSRTrainer, self).__init__()
        self.config = config
        self.set_params()

        self.training = True  # training mode by default

    def set_params_replica(self):
        self.H = self.config["experiment"]["height"]
        self.W = self.config["experiment"]["width"]

        self.n_pix = self.H * self.W
        self.aspect_ratio = self.W/self.H

        self.hfov = 90
        # the pin-hole camera has the same value for fx and fy
        self.fx = self.W / 2.0 / math.tan(math.radians(self.hfov / 2.0))
        # self.fy = self.H / 2.0 / math.tan(math.radians(self.yhov / 2.0))
        self.fy = self.fx
        self.cx = (self.W - 1.0) / 2.0
        self.cy = (self.H - 1.0) / 2.0
        self.near, self.far = self.config["render"]["depth_range"]
        self.c2w_staticcam = None

        # use scaled size for test and visualisation purpose
        self.test_viz_factor = int(self.config["render"]["test_viz_factor"])
        self.H_scaled = self.H//self.test_viz_factor
        self.W_scaled = self.W//self.test_viz_factor
        self.fx_scaled = self.W_scaled / 2.0 / math.tan(math.radians(self.hfov / 2.0))
        # self.fy_scaled = self.H_scaled / 2.0 / math.tan(math.radians(self.yhov / 2.0))
        self.fy_scaled = self.fx_scaled
        self.cx_scaled = (self.W_scaled - 1.0) / 2.0
        self.cy_scaled = (self.H_scaled - 1.0) / 2.0


    def set_params(self):
        self.enable_semantic = self.config["experiment"]["enable_semantic"]

        #render options
        self.n_rays = eval(self.config["render"]["N_rays"])  if isinstance(self.config["render"]["N_rays"], str) \
            else self.config["render"]["N_rays"]

        self.N_samples = self.config["render"]["N_samples"]
        self.netchunk = eval(self.config["model"]["netchunk"]) if isinstance(self.config["model"]["netchunk"], str) \
            else self.config["model"]["netchunk"]

        self.chunk = eval(self.config["model"]["chunk"])  if isinstance(self.config["model"]["chunk"], str) \
            else self.config["model"]["chunk"]

        self.use_viewdir = self.config["render"]["use_viewdirs"]

        self.convention = self.config["experiment"]["convention"]

        self.endpoint_feat = self.config["experiment"]["endpoint_feat"] if "endpoint_feat" in self.config["experiment"].keys() else False

        self.N_importance = self.config["render"]["N_importance"]
        self.raw_noise_std = self.config["render"]["raw_noise_std"]
        self.white_bkgd = self.config["render"]["white_bkgd"]
        self.perturb = self.config["render"]["perturb"]

        self.no_batching = self.config["render"]["no_batching"]

        self.lrate = float(self.config["train"]["lrate"])
        self.lrate_decay = float(self.config["train"]["lrate_decay"])

        # logging
        self.save_dir = self.config["experiment"]["save_dir"]

    def prepare_data_replica(self, data, gpu=True):
        self.ignore_label = -1

        # shift numpy data to torch
        train_samples = data.train_samples
        test_samples = data.test_samples

        self.train_ids = data.train_ids
        self.test_ids = data.test_ids
        self.mask_ids = data.mask_ids

        self.num_train = data.train_num
        self.num_test = data.test_num

        # preprocess semantic info
        self.semantic_classes = torch.from_numpy(data.semantic_classes)
        print(self.semantic_classes)
        self.num_semantic_class = self.semantic_classes.shape[0]  # number of semantic classes, including void class=0
        self.num_valid_semantic_class = self.num_semantic_class - 1  # exclude void class
        assert self.num_semantic_class==data.num_semantic_class

        json_class_mapping = "/content/semantic_info/office_2/info_semantic.json"
        with open(json_class_mapping, "r") as f:
            annotations = json.load(f)
        instance_id_to_semantic_label_id = np.array(annotations["id_to_label"])
        total_num_classes = len(annotations["classes"])
        assert total_num_classes==101  # excluding void we have 102 classes
        # assert self.num_valid_semantic_class == np.sum(np.unique(instance_id_to_semantic_label_id) >=0 )

        colour_map_np = label_colormap(total_num_classes)[data.semantic_classes] # select the existing class from total colour map
        self.colour_map = torch.from_numpy(colour_map_np)
        self.valid_colour_map  = torch.from_numpy(colour_map_np[1:,:]) # exclude the first colour map to colourise rendered segmentation without void index

        # plot semantic label legend
        # class_name_string = ["voild"] + [x["name"] for x in annotations["classes"] if x["id"] in np.unique(data.semantic)]
        # class_name_string = ["void"] + [x["name"] for x in annotations["classes"]]
        # legend_img_arr = image_utils.plot_semantic_legend(data.semantic_classes, class_name_string,
        # colormap=label_colormap(total_num_classes+1), save_path=self.save_dir)
        # total_num_classes +1 to include void class

        # remap different semantic classes to continuous integers from 0 to num_class-1
        self.semantic_classes_remap = torch.from_numpy(np.arange(self.num_semantic_class))

        #####training data#####
        # rgb
        self.train_image = torch.from_numpy(train_samples["image"])
        self.train_image_scaled = F.interpolate(self.train_image.permute(0,3,1,2,),
                                    scale_factor=1/self.config["render"]["test_viz_factor"],
                                    mode='bilinear').permute(0,2,3,1)
        # depth
        self.train_depth = torch.from_numpy(train_samples["depth"])
        self.viz_train_depth = np.stack([depth2rgb(dep, min_value=self.near, max_value=self.far) for dep in train_samples["depth"]], axis=0) # [num_test, H, W, 3]
        # process the depth for evaluation purpose
        self.train_depth_scaled = F.interpolate(torch.unsqueeze(self.train_depth, dim=1).float(),
                                                            scale_factor=1/self.config["render"]["test_viz_factor"],
                                                            mode='bilinear').squeeze(1).cpu().numpy()

        # semantic
        self.train_semantic = torch.from_numpy(train_samples["semantic_remap"])
        self.viz_train_semantic = np.stack([colour_map_np[sem] for sem in self.train_semantic], axis=0) # [num_test, H, W, 3]

        self.train_semantic_clean = torch.from_numpy(train_samples["semantic_remap_clean"])
        self.viz_train_semantic_clean = np.stack([colour_map_np[sem] for sem in self.train_semantic_clean], axis=0) # [num_test, H, W, 3]

        # process the clean label for evaluation purpose
        self.train_semantic_clean_scaled = F.interpolate(torch.unsqueeze(self.train_semantic_clean, dim=1).float(),
                                                            scale_factor=1/self.config["render"]["test_viz_factor"],
                                                            mode='nearest').squeeze(1)
        self.train_semantic_clean_scaled = self.train_semantic_clean_scaled.cpu().numpy() - 1
        # pose
        self.train_Ts = torch.from_numpy(train_samples["T_wc"]).float()


        #####test data#####
        # rgb
        self.test_image = torch.from_numpy(test_samples["image"])  # [num_test, H, W, 3]
        # scale the test image for evaluation purpose
        self.test_image_scaled = F.interpolate(self.test_image.permute(0,3,1,2,),
                                            scale_factor=1/self.config["render"]["test_viz_factor"],
                                            mode='bilinear').permute(0,2,3,1)


        # depth
        self.test_depth = torch.from_numpy(test_samples["depth"])  # [num_test, H, W]
        self.viz_test_depth = np.stack([depth2rgb(dep, min_value=self.near, max_value=self.far) for dep in test_samples["depth"]], axis=0) # [num_test, H, W, 3]
        # process the depth for evaluation purpose
        self.test_depth_scaled = F.interpolate(torch.unsqueeze(self.test_depth, dim=1).float(),
                                                            scale_factor=1/self.config["render"]["test_viz_factor"],
                                                            mode='bilinear').squeeze(1).cpu().numpy()
        # semantic
        self.test_semantic = torch.from_numpy(test_samples["semantic_remap"])  # [num_test, H, W]

        self.viz_test_semantic = np.stack([colour_map_np[sem] for sem in self.test_semantic], axis=0) # [num_test, H, W, 3]

        # we only add noise to training images, therefore test images are kept intact. No need for test_remap_clean
        # process the clean label for evaluation purpose
        self.test_semantic_scaled = F.interpolate(torch.unsqueeze(self.test_semantic, dim=1).float(),
                                                    scale_factor=1/self.config["render"]["test_viz_factor"],
                                                    mode='nearest').squeeze(1)
        self.test_semantic_scaled = self.test_semantic_scaled.cpu().numpy() - 1 # shift void class from value 0 to -1, to match self.ignore_label
        # pose
        self.test_Ts = torch.from_numpy(test_samples["T_wc"]).float()  # [num_test, 4, 4]

        if gpu is True:
            self.train_image = self.train_image.cuda()
            self.train_image_scaled = self.train_image_scaled.cuda()
            self.train_depth = self.train_depth.cuda()
            self.train_semantic = self.train_semantic.cuda()

            self.test_image = self.test_image.cuda()
            self.test_image_scaled = self.test_image_scaled.cuda()
            self.test_depth = self.test_depth.cuda()
            self.test_semantic = self.test_semantic.cuda()
            self.colour_map = self.colour_map.cuda()
            self.valid_colour_map = self.valid_colour_map.cuda()


        # set the data sampling paras which need the number of training images
        if self.no_batching is False: # False means we need to sample from all rays instead of rays from one random image
            self.i_batch = 0
            self.rand_idx = torch.randperm(self.num_train*self.H*self.W)

    def init_rays(self):

        # create rays
        rays = create_rays(self.num_train, self.train_Ts, self.H, self.W, self.fx, self.fy, self.cx, self.cy,
                                self.near, self.far, use_viewdirs=self.use_viewdir, convention=self.convention)

        rays_vis = create_rays(self.num_train, self.train_Ts, self.H_scaled, self.W_scaled, self.fx_scaled, self.fy_scaled,
                            self.cx_scaled, self.cy_scaled, self.near, self.far, use_viewdirs=self.use_viewdir, convention=self.convention)


        rays_test = create_rays(self.num_test, self.test_Ts, self.H_scaled, self.W_scaled, self.fx_scaled, self.fy_scaled,
                                self.cx_scaled, self.cy_scaled, self.near, self.far, use_viewdirs=self.use_viewdir, convention=self.convention)

        # init rays
        self.rays = rays.cuda() # [num_images, H*W, 11]
        self.rays_vis = rays_vis.cuda()
        self.rays_test = rays_test.cuda()

    def new_viewpoints_rays(self, n_rays=180):
      camera_poses =  generate_circle_around_center([3,3,1], 1.0, num_poses=n_rays)
      rays = create_rays(n_rays, torch.tensor(camera_poses).float(), self.H_scaled, self.W_scaled, self.fx_scaled, self.fy_scaled,
                                self.cx_scaled, self.cy_scaled, self.near, self.far, use_viewdirs=self.use_viewdir, convention=self.convention)
      self.new_rays = rays.cuda()


    def sample_data(self, step, rays, h, w, no_batching=True, mode="train"):
        # generate sampling index
        num_img, num_ray, ray_dim = rays.shape

        assert num_ray == h*w
        total_ray_num = num_img*h*w

        if mode == "train":
            image = self.train_image
            if self.enable_semantic:
                depth = self.train_depth
                semantic = self.train_semantic
            sample_num = self.num_train
        elif mode == "test":
            image = self.test_image
            if self.enable_semantic:
                depth = self.test_depth
                semantic = self.test_semantic
            sample_num = self.num_test
        elif mode == "vis":
            assert False
        else:
            assert False

        # sample rays and ground truth data
        sematic_available_flag = 1

        if no_batching:  # sample random pixels from one random images
            index_batch = np.random.choice(np.arange(num_img)).reshape((1, 1))  # sample one image from the full trainiing set
            index_hw = torch.randint(0, h * w, (1, self.n_rays))
            sampled_rays = rays[index_batch, index_hw, :]

            flat_sampled_rays = sampled_rays.reshape([-1, ray_dim]).float()
            gt_image = image.reshape(sample_num, -1, 3)[index_batch, index_hw, :].reshape(-1, 3)
            if self.enable_semantic:
                gt_depth = depth.reshape(sample_num, -1)[index_batch, index_hw].reshape(-1)
                sematic_available_flag = self.mask_ids[index_batch] # semantic available if mask_id is 1 (train with rgb loss and semantic loss) else 0 (train with rgb loss only)
                gt_semantic = semantic.reshape(sample_num, -1)[index_batch, index_hw].reshape(-1)
                gt_semantic = gt_semantic.cuda()
        else:  # sample from all random pixels

            index_hw = self.rand_idx[self.i_batch:self.i_batch+self.n_rays]

            flat_rays = rays.reshape([-1, ray_dim]).float()
            flat_sampled_rays = flat_rays[index_hw, :]
            gt_image = image.reshape(-1, 3)[index_hw, :]
            if self.enable_semantic:
                gt_depth = depth.reshape(-1)[index_hw]
                gt_semantic = semantic.reshape(-1)[index_hw]
                gt_semantic = gt_semantic.cuda()

            self.i_batch += self.n_rays
            if self.i_batch >= total_ray_num:
                print("Shuffle data after an epoch!")
                self.rand_idx = torch.randperm(total_ray_num)
                self.i_batch = 0

        sampled_rays = flat_sampled_rays
        sampled_gt_rgb = gt_image
        if self.enable_semantic:
            sampled_gt_depth = gt_depth

            sampled_gt_semantic = gt_semantic.long()  # required long type for nn.NLL or nn.crossentropy

            return sampled_rays, sampled_gt_rgb, sampled_gt_depth, sampled_gt_semantic, sematic_available_flag
        else:
            return sampled_rays, sampled_gt_rgb

    def render_rays(self, flat_rays):
        """
        Render rays, run in optimisation loop
        Returns:
          List of:
            rgb_map: [batch_size, 3]. Predicted RGB values for rays.
            disp_map: [batch_size]. Disparity map. Inverse of depth.
            acc_map: [batch_size]. Accumulated opacity (alpha) along a ray.
          Dict of extras: dict with everything returned by render_rays().
        """

        # Render and reshape
        ray_shape = flat_rays.shape  # num_rays, 11

        # assert ray_shape[0] == self.n_rays  # this is not satisfied in test model
        fn = self.volumetric_rendering
        all_ret = batchify_rays(fn, flat_rays, self.chunk)

        for k in all_ret:
            k_sh = list(ray_shape[:-1]) + list(all_ret[k].shape[1:])
            all_ret[k] = torch.reshape(all_ret[k], k_sh)

        return all_ret


    def render_rays_chunk(self, flat_rays, chunk_size=1024*4):
        """Render rays while moving resulting chunks to cpu to avoid OOM when rendering large images."""
        B = flat_rays.shape[0]  # num_rays
        results = defaultdict(list)
        for i in range(0, B, chunk_size):
            rendered_ray_chunks = \
                self.render_rays(flat_rays[i:i+chunk_size])

            for k, v in rendered_ray_chunks.items():
                results[k] += [v.cpu()]

        for k, v in results.items():
            results[k] = torch.cat(v, 0)
        return results


    def volumetric_rendering(self, ray_batch):
        """
        Volumetric Rendering
        """
        N_rays = ray_batch.shape[0]

        rays_o, rays_d = ray_batch[:, 0:3], ray_batch[:, 3:6]  # [N_rays, 3] each
        viewdirs = ray_batch[:, -3:] if ray_batch.shape[-1] > 8 else None


        bounds = torch.reshape(ray_batch[..., 6:8], [-1, 1, 2])
        near, far = bounds[..., 0], bounds[..., 1]  # [N_rays, 1], [N_rays, 1]

        #pts_coarse_sampled = sample_stratified(rays_o,rays_d,near,far,self.N_samples,self.perturb > 0. and self.training,False)

        t_vals = torch.linspace(0., 1., steps=self.N_samples).cuda()

        z_vals = near * (1. - t_vals) + far * (t_vals) # use linear sampling in depth space


        z_vals = z_vals.expand([N_rays, self.N_samples])

        if self.perturb > 0. and self.training:  # perturb sampling depths (z_vals)
            if self.training is True:  # only add perturbation during training intead of testing
                # get intervals between samples
                mids = .5 * (z_vals[..., 1:] + z_vals[..., :-1])
                upper = torch.cat([mids, z_vals[..., -1:]], -1)
                lower = torch.cat([z_vals[..., :1], mids], -1)
                # stratified samples in those intervals
                t_rand = torch.rand(z_vals.shape).cuda()

                z_vals = lower + (upper - lower) * t_rand

        pts_coarse_sampled = rays_o[..., None, :] + rays_d[..., None, :] * z_vals[..., :, None]  # [N_rays, N_samples, 3]

        raw_noise_std = self.raw_noise_std if self.training else 0
        raw_coarse = run_network(pts_coarse_sampled, viewdirs, self.ssr_net_coarse,
                                 self.embed_fn, self.embeddirs_fn, netchunk=self.netchunk)
        rgb_coarse, disp_coarse, acc_coarse, weights_coarse, depth_coarse, sem_logits_coarse, feat_map_coarse = \
            raw2outputs(raw_coarse, z_vals, rays_d, raw_noise_std, self.white_bkgd, enable_semantic = self.enable_semantic,
            num_sem_class = self.num_valid_semantic_class, endpoint_feat = False)


        if self.N_importance > 0:
            z_vals_mid = .5 * (z_vals[..., 1:] + z_vals[..., :-1])  # (N_rays, N_samples-1) interval mid points
            z_samples = sample_pdf(z_vals_mid, weights_coarse[..., 1:-1], self.N_importance,
                                   det=(self.perturb == 0.) or (not self.training))
            z_samples = z_samples.detach()
            # detach so that grad doesn't propogate to weights_coarse from here
            # values are interleaved actually, so maybe can do better than sort?

            z_vals, _ = torch.sort(torch.cat([z_vals, z_samples], -1), -1)
            pts_fine_sampled = rays_o[..., None, :] + rays_d[..., None, :] * z_vals[..., :, None]  # [N_rays, N_samples + N_importance, 3]


            raw_fine = run_network(pts_fine_sampled, viewdirs, lambda x: self.ssr_net_fine(x, self.endpoint_feat),
                        self.embed_fn, self.embeddirs_fn, netchunk=self.netchunk)

            rgb_fine, disp_fine, acc_fine, weights_fine, depth_fine, sem_logits_fine, feat_map_fine = \
                raw2outputs(raw_fine, z_vals, rays_d, raw_noise_std, self.white_bkgd, enable_semantic = self.enable_semantic,
                num_sem_class = self.num_valid_semantic_class, endpoint_feat = self.endpoint_feat)

        ret = {}
        ret['raw_coarse'] = raw_coarse
        ret['rgb_coarse'] = rgb_coarse
        ret['disp_coarse'] = disp_coarse
        ret['acc_coarse'] = acc_coarse
        ret['depth_coarse'] = depth_coarse
        if self.enable_semantic:
            ret['sem_logits_coarse'] = sem_logits_coarse

        if self.N_importance > 0:
            ret['rgb_fine'] = rgb_fine
            ret['disp_fine'] = disp_fine
            ret['acc_fine'] = acc_fine
            ret['depth_fine'] = depth_fine
            if self.enable_semantic:
                ret['sem_logits_fine'] = sem_logits_fine
            ret['z_std'] = torch.std(z_samples, dim=-1, unbiased=False)  # [N_rays]
            ret['raw_fine'] = raw_fine  # model's raw, unprocessed predictions.
            if self.endpoint_feat:
                ret['feat_map_fine'] = feat_map_fine
        for k in ret:
            # if (torch.isnan(ret[k]).any() or torch.isinf(ret[k]).any()) and self.config["experiment"]["debug"]:
            if (torch.isnan(ret[k]).any() or torch.isinf(ret[k]).any()):
                print(f"! [Numerical Error] {k} contains nan or inf.")

        return ret


    def create_ssr(self):
        """Instantiate NeRF's MLP model.
        """

        nerf_model = Semantic_NeRF
        embed_fn = PositionalEncoder(3, self.config["render"]["multires"], True)
        input_ch = embed_fn.d_output
        #embed_fn, input_ch = get_embedder(self.config["render"]["multires"], self.config["render"]["i_embed"], scalar_factor=10)
        input_ch_views = 0
        embeddirs_fn = None
        if self.config["render"]["use_viewdirs"]:
            # embeddirs_fn, input_ch_views = get_embedder(self.config["render"]["multires_views"],
            #                                             self.config["render"]["i_embed"],
            #                                             scalar_factor=1)
            embeddirs_fn = PositionalEncoder(3, self.config["render"]["multires_views"], True)
            input_ch_views = embeddirs_fn.d_output
        output_ch = 5 if self.N_importance > 0 else 4
        skips = [4]
        print("Number of semantic classes: ",self.num_valid_semantic_class)
        model = nerf_model(enable_semantic = self.enable_semantic, num_semantic_classes=self.num_valid_semantic_class,
                     D=self.config["model"]["netdepth"], W=self.config["model"]["netwidth"],
                     input_ch=input_ch, output_ch=output_ch, skips=skips,
                     input_ch_views=input_ch_views, use_viewdirs=self.config["render"]["use_viewdirs"]).cuda()
        grad_vars = list(model.parameters())

        model_fine = None
        if self.N_importance > 0:
            model_fine = nerf_model(enable_semantic = self.enable_semantic, num_semantic_classes=self.num_valid_semantic_class,
                              D=self.config["model"]["netdepth_fine"], W=self.config["model"]["netwidth_fine"],
                              input_ch=input_ch, output_ch=output_ch, skips=skips,
                              input_ch_views=input_ch_views, use_viewdirs=self.config["render"]["use_viewdirs"]).cuda()
            grad_vars += list(model_fine.parameters())

        # Create optimizer
        optimizer = torch.optim.Adam(params=grad_vars, lr=self.lrate)

        self.ssr_net_coarse = model
        self.ssr_net_fine = model_fine
        self.embed_fn = embed_fn
        self.embeddirs_fn = embeddirs_fn
        self.optimizer = optimizer

    # optimisation step
    def step(
        self,
        global_step
    ):
        # Misc
        img2mse = lambda x, y: torch.mean((x - y) ** 2)
        mse2psnr = lambda x: -10. * torch.log(x) / torch.log(torch.Tensor([10.]).cuda())
        CrossEntropyLoss = nn.CrossEntropyLoss(ignore_index=self.ignore_label)
        KLDLoss = nn.KLDivLoss(reduction="none")
        kl_loss = lambda input_log_prob,target_prob: KLDLoss(input_log_prob, target_prob)
        # this function assume input is already in log-probabilities

        crossentropy_loss = lambda logit, label: CrossEntropyLoss(logit, label-1)  # replica has void class of ID==0, label-1 to shift void class to -1

        logits_2_label = lambda x: torch.argmax(torch.nn.functional.softmax(x, dim=-1),dim=-1)
        logits_2_prob = lambda x: F.softmax(x, dim=-1)
        to8b_np = lambda x: (255 * np.clip(x, 0, 1)).astype(np.uint8)
        to8b = lambda x: (255 * torch.clamp(x, 0, 1)).type(torch.uint8)


        # sample rays to query and optimise
        sampled_data = self.sample_data(global_step, self.rays, self.H, self.W, no_batching=True, mode="train")
        if self.enable_semantic:
            sampled_rays, sampled_gt_rgb, sampled_gt_depth, sampled_gt_semantic, sematic_available = sampled_data
        else:
            sampled_rays, sampled_gt_rgb = sampled_data

        output_dict = self.render_rays(sampled_rays)

        rgb_coarse = output_dict["rgb_coarse"]  # N_rays x 3
        disp_coarse = output_dict["disp_coarse"] # N_rays
        depth_coarse = output_dict["depth_coarse"] # N_rays
        acc_coarse = output_dict["acc_coarse"] # N_rays
        if self.enable_semantic:
            sem_logits_coarse = output_dict["sem_logits_coarse"] # N_rays x num_classes
            sem_label_coarse = logits_2_label(sem_logits_coarse)  # N_rays

        if self.N_importance > 0:
            rgb_fine = output_dict["rgb_fine"]
            disp_fine = output_dict["disp_fine"]
            depth_fine = output_dict["depth_fine"]
            acc_fine = output_dict["acc_fine"]
            z_std = output_dict["z_std"] # N_rays
            if self.enable_semantic:
                sem_logits_fine = output_dict["sem_logits_fine"]
                sem_label_fine = logits_2_label(sem_logits_fine)

        self.optimizer.zero_grad()

        img_loss_coarse = img2mse(rgb_coarse, sampled_gt_rgb)

        if self.enable_semantic and sematic_available:
            # print(sem_logits_coarse.shape, sampled_gt_semantic.shape)
            # print(sampled_gt_semantic)
            # print(sem_logits_coarse)
            semantic_loss_coarse = crossentropy_loss(sem_logits_coarse, sampled_gt_semantic)
        else:
            semantic_loss_coarse = torch.tensor(0)


        with torch.no_grad():
            psnr_coarse = mse2psnr(img_loss_coarse)

        if self.N_importance > 0:
            img_loss_fine = img2mse(rgb_fine, sampled_gt_rgb)
            if self.enable_semantic and sematic_available:
                semantic_loss_fine = crossentropy_loss(sem_logits_fine, sampled_gt_semantic)
            else:
                semantic_loss_fine = torch.tensor(0)
            with torch.no_grad():
                psnr_fine = mse2psnr(img_loss_fine)
        else:
            img_loss_fine = torch.tensor(0)
            psnr_fine = torch.tensor(0)

        total_img_loss = img_loss_coarse + img_loss_fine
        total_sem_loss = semantic_loss_coarse + semantic_loss_fine


        wgt_sem_loss = float(self.config["train"]["wgt_sem"])
        total_loss = total_img_loss + total_sem_loss*wgt_sem_loss

        total_loss.backward()
        self.optimizer.step()

        ###   update learning rate   ###
        decay_rate = 0.1
        decay_steps = self.lrate_decay
        new_lrate = self.lrate * (decay_rate ** (global_step / decay_steps))
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = new_lrate
        ################################

        # Rest is logging
        # saving ckpts regularly
        if global_step % float(self.config["logging"]["step_save_ckpt"])==0:
            ckpt_dir = os.path.join(self.save_dir, "checkpoints")
            if not os.path.exists(ckpt_dir):
                os.makedirs(ckpt_dir)

            ckpt_file = os.path.join(ckpt_dir, '{:06d}.ckpt'.format(global_step))
            torch.save({
                'global_step': global_step,
                'network_coarse_state_dict': self.ssr_net_coarse.state_dict(),
                'network_fine_state_dict': self.ssr_net_fine.state_dict(),
                'optimizer_state_dict': self.optimizer.state_dict(),
            }, ckpt_file)
            print('Saved checkpoints at', ckpt_file)

        # render and save training-set images
        if global_step % self.config["logging"]["step_vis_train"]==0 and global_step > 0:
            self.training = False  # enable testing mode before rendering results, need to set back during training!
            self.ssr_net_coarse.eval()
            self.ssr_net_fine.eval()
            trainsavedirvideo = os.path.join(self.config["experiment"]["save_dir"], "train_render_video", f"step_{global_step}")
            trainsavedir = os.path.join(self.config["experiment"]["save_dir"], "train_render", 'step_{:06d}'.format(global_step))
            os.makedirs(trainsavedir, exist_ok=True)
            os.makedirs(trainsavedirvideo, exist_ok=True)
            print(' {} train images'.format(self.num_train))
            with torch.no_grad():
                # rgbs, disps, deps, vis_deps, sems, vis_sems
                rgbs, disps, deps, vis_deps, sems, vis_sems, sem_uncers, vis_sem_uncers = self.render_path(self.rays_vis, save_dir=trainsavedir)
                #  numpy array of shape [B,H,W,C] or [B,H,W]
            print('Saved training set')

            self.training = True  # set training flag back after rendering images
            self.ssr_net_coarse.train()
            self.ssr_net_fine.train()

            with torch.no_grad():
                if self.enable_semantic:
                    # mask out void regions for better visualisation
                    for idx in range(vis_sems.shape[0]):
                        vis_sems[idx][self.train_semantic_clean_scaled[idx]==self.ignore_label,:] = 0

                batch_train_img_mse = img2mse(torch.from_numpy(rgbs), self.train_image_scaled.cpu())
                batch_train_img_psnr = mse2psnr(batch_train_img_mse)

            imageio.mimwrite(os.path.join(trainsavedirvideo, 'rgb.mp4'), to8b_np(rgbs), fps=30, quality=8)
            imageio.mimwrite(os.path.join(trainsavedirvideo, 'dep.mp4'), vis_deps, fps=30, quality=8)
            imageio.mimwrite(os.path.join(trainsavedirvideo, 'disps.mp4'), to8b_np(disps / np.max(disps)), fps=30, quality=8)
            if self.enable_semantic:
                imageio.mimwrite(os.path.join(trainsavedirvideo, 'sem.mp4'), vis_sems, fps=30, quality=8)
                imageio.mimwrite(os.path.join(trainsavedirvideo, 'sem_uncertainty.mp4'), vis_sem_uncers, fps=30, quality=8)


            if self.enable_semantic:
                miou_train, miou_train_validclass ,total_accuracy_train, class_average_accuracy_train, ious_train = \
                calculate_segmentation_metrics(true_labels=self.train_semantic_clean_scaled, predicted_labels=sems,
                number_classes=self.num_valid_semantic_class, ignore_label=self.ignore_label)

                tqdm.write(f"[Training Metric] Iter: {global_step} "
                       f"img_loss: {total_img_loss.item()}, semantic_loss: {(total_sem_loss*wgt_sem_loss).item()},"
                       f"psnr_coarse: {psnr_coarse.item()}, psnr_fine: {psnr_fine.item()},"
                       f"mIoU: {miou_train}, total_acc: {total_accuracy_train}, avg_acc: {class_average_accuracy_train}")


        # render and save test images, corresponding videos
        if global_step % self.config["logging"]["step_val"]==0 and global_step > 0:
            self.training = False  # enable testing mode before rendering results, need to set back during training!
            self.ssr_net_coarse.eval()
            self.ssr_net_fine.eval()
            testsavedirvideo = os.path.join(self.config["experiment"]["save_dir"], "test_render_video", 'step_{:06d}'.format(global_step))
            testsavedir = os.path.join(self.config["experiment"]["save_dir"], "test_render", 'step_{:06d}'.format(global_step))
            os.makedirs(testsavedir, exist_ok=True)
            os.makedirs(testsavedirvideo, exist_ok=True)
            print(' {} test images'.format(self.num_test))
            with torch.no_grad():
                rgbs, disps, deps, vis_deps, sems, vis_sems, sem_uncers, vis_sem_uncers = self.render_path(self.rays_test, save_dir=testsavedir)
            print('Saved test set')


            self.training = True  # set training flag back after rendering images
            self.ssr_net_coarse.train()
            self.ssr_net_fine.train()


            with torch.no_grad():
                if self.enable_semantic:
                    # mask out void regions for better visualisation
                    for idx in range(vis_sems.shape[0]):
                        vis_sems[idx][self.test_semantic_scaled[idx]==self.ignore_label, :] = 0

                batch_test_img_mse = img2mse(torch.from_numpy(rgbs), self.test_image_scaled.cpu())
                batch_test_img_psnr = mse2psnr(batch_test_img_mse)

            imageio.mimwrite(os.path.join(testsavedirvideo, 'rgb.mp4'), to8b_np(rgbs), fps=30, quality=8)
            imageio.mimwrite(os.path.join(testsavedirvideo, 'dep.mp4'), vis_deps, fps=30, quality=8)
            imageio.mimwrite(os.path.join(testsavedirvideo, 'disps.mp4'), to8b_np(disps / np.max(disps)), fps=30, quality=8)
            if self.enable_semantic:
                imageio.mimwrite(os.path.join(testsavedirvideo, 'sem.mp4'), vis_sems, fps=30, quality=8)
                imageio.mimwrite(os.path.join(testsavedirvideo, 'sem_uncertainty.mp4'), vis_sem_uncers, fps=30, quality=8)

            if self.enable_semantic:

                # add segmentation quantative metrics during testung into tfb
                miou_test, miou_test_validclass, total_accuracy_test, class_average_accuracy_test, ious_test = \
                calculate_segmentation_metrics(true_labels=self.test_semantic_scaled, predicted_labels=sems,
                number_classes=self.num_valid_semantic_class, ignore_label=self.ignore_label)

        if global_step%self.config["logging"]["step_log_print"]==0:
            tqdm.write(f"[TRAIN] Iter: {global_step} "
                       f"Loss: {total_loss.item()} "
                       f"rgb_total_loss: {total_img_loss.item()}, rgb_coarse: {img_loss_coarse.item()}, rgb_fine: {img_loss_fine.item()}, "
                       f"sem_total_loss: {total_sem_loss.item()}, weighted_sem_total_loss: {total_sem_loss.item()*wgt_sem_loss}, "
                       f"semantic_loss: {semantic_loss_coarse.item()}, semantic_fine: {semantic_loss_fine.item()}, "
                       f"PSNR_coarse: {psnr_coarse.item()}, PSNR_fine: {psnr_fine.item()}",)


    def load_models(self, path):

      checkpoint = torch.load(path)
      self.ssr_net_coarse.load_state_dict(checkpoint["network_coarse_state_dict"])
      self.ssr_net_fine.load_state_dict(checkpoint["network_fine_state_dict"])

    def new_paths(self, path):
      to8b_np = lambda x: (255 * np.clip(x, 0, 1)).astype(np.uint8)
      self.load_models(path)

      self.training = False  # enable testing mode before rendering results, need to set back during training!
      self.ssr_net_coarse.eval()
      self.ssr_net_fine.eval()
      testsavedirRGB = os.path.join(self.config["experiment"]["save_dir"], 'training_cnn', 'rgb')
      testsavedirLabel = os.path.join(self.config["experiment"]["save_dir"], 'training_cnn', 'label')
      os.makedirs(testsavedirRGB, exist_ok=True)
      os.makedirs(testsavedirLabel, exist_ok=True)
      print(' {} test images'.format(self.num_test))
      with torch.no_grad():
          rgbs, vis_sems, sems = self.render_path_for_training(self.new_rays, save_dir_rgbs=testsavedirRGB, save_dir_labels = testsavedirLabel)
          imageio.mimwrite(os.path.join(testsavedirRGB, 'rgb.mp4'), to8b_np(rgbs), fps=30, quality=8)
          imageio.mimwrite(os.path.join(testsavedirLabel, 'vis_sems.mp4'), vis_sems, fps=30, quality=8)
          print(vis_sems.shape)
      print('Saved test set')

    def render_path(self, rays, save_dir=None):

        rgbs = []
        disps = []

        deps = []
        vis_deps = []

        sems = []
        vis_sems = []

        entropys = []
        vis_entropys = []

        to8b_np = lambda x: (255 * np.clip(x, 0, 1)).astype(np.uint8)
        to8b = lambda x: (255 * torch.clamp(x, 0, 1)).type(torch.uint8)
        logits_2_label = lambda x: torch.argmax(torch.nn.functional.softmax(x, dim=-1),dim=-1)
        logits_2_uncertainty = lambda x: torch.sum(-F.log_softmax(x, dim=-1)*F.softmax(x, dim=-1), dim=-1, keepdim=True)

        t = time.time()
        for i, c2w in enumerate(tqdm(rays)):
            print(i, time.time() - t)
            t = time.time()
            output_dict = self.render_rays_chunk(rays[i], self.chunk) # render rays by chunks to avoid OOM
            rgb_coarse = output_dict["rgb_coarse"]
            disp_coarse = output_dict["disp_coarse"]
            depth_coarse = output_dict["depth_coarse"]

            rgb = rgb_coarse
            depth =depth_coarse
            disp = disp_coarse

            if self.enable_semantic:
                sem_label_coarse = logits_2_label(output_dict["sem_logits_coarse"])
                sem_uncertainty_coarse = logits_2_uncertainty(output_dict["sem_logits_coarse"])
                vis_sem_label_coarse = self.valid_colour_map[sem_label_coarse]
                # shift pred label by +1 to shift the first valid class to use the first valid colour-map instead of the void colour-map
                sem_label = sem_label_coarse
                vis_sem = vis_sem_label_coarse
                sem_uncertainty = sem_uncertainty_coarse


            if self.N_importance > 0:
                rgb_fine = output_dict["rgb_fine"]
                depth_fine= output_dict["depth_fine"]
                disp_fine = output_dict["disp_fine"]

                rgb = rgb_fine
                depth = depth_fine
                disp = disp_fine

                if self.enable_semantic:
                    sem_label_fine = logits_2_label(output_dict["sem_logits_fine"])
                    sem_uncertainty_fine = logits_2_uncertainty(output_dict["sem_logits_fine"])
                    vis_sem_label_fine = self.valid_colour_map[sem_label_fine]
                    # shift pred label by +1 to shift the first valid class to use the first valid colour-map instead of the void colour-map
                    sem_label = sem_label_fine
                    vis_sem = vis_sem_label_fine
                    sem_uncertainty = sem_uncertainty_fine

            rgb = rgb.cpu().numpy().reshape((self.H_scaled, self.W_scaled, 3))
            depth = depth.cpu().numpy().reshape((self.H_scaled, self.W_scaled))
            disp = disp.cpu().numpy().reshape((self.H_scaled, self.W_scaled))

            rgbs.append(rgb)
            disps.append(disp)

            deps.append(depth)  # save depth in mm
            vis_deps.append(depth2rgb(depth, min_value=self.near, max_value=self.far))


            if self.enable_semantic:
                sem_label = sem_label.cpu().numpy().astype(np.uint8).reshape((self.H_scaled, self.W_scaled))
                vis_sem = vis_sem.cpu().numpy().astype(np.uint8).reshape((self.H_scaled, self.W_scaled, 3))
                sem_uncertainty = sem_uncertainty.cpu().numpy().reshape((self.H_scaled, self.W_scaled))
                vis_sem_uncertainty = depth2rgb(sem_uncertainty)

                sems.append(sem_label)
                vis_sems.append(vis_sem)

                entropys.append(sem_uncertainty)
                vis_entropys.append(vis_sem_uncertainty)

            if i==0:
                print(rgb.shape, disp.shape)

            if save_dir is not None:
                assert os.path.exists(save_dir)
                rgb8 = to8b_np(rgbs[-1])
                disp = disps[-1].astype(np.uint16)
                dep_mm = (deps[-1]*1000).astype(np.uint16)
                vis_dep = vis_deps[-1]


                rgb_filename = os.path.join(save_dir, 'rgb_{:03d}.png'.format(i))
                disp_filename = os.path.join(save_dir, 'disp_{:03d}.png'.format(i))

                depth_filename = os.path.join(save_dir, 'depth_{:03d}.png'.format(i))
                vis_depth_filename = os.path.join(save_dir, 'vis_depth_{:03d}.png'.format(i))

                imageio.imwrite(rgb_filename, rgb8)
                imageio.imwrite(disp_filename, disp, format="png", prefer_uint8=False)

                imageio.imwrite(depth_filename, dep_mm, format="png", prefer_uint8=False)
                imageio.imwrite(vis_depth_filename, vis_dep)

                if self.enable_semantic:

                    label_filename = os.path.join(save_dir, 'label_{:03d}.png'.format(i))
                    vis_label_filename = os.path.join(save_dir, 'vis_label_{:03d}.png'.format(i))

                    entropy_filename = os.path.join(save_dir, 'entropy_{:03d}.png'.format(i))
                    vis_entropy_filename = os.path.join(save_dir, 'vis_entropy_{:03d}.png'.format(i))
                    sem = sems[-1]
                    vis_sem = vis_sems[-1]


                    sem_uncer = to8b_np(entropys[-1])
                    vis_sem_uncer = vis_entropys[-1]

                    imageio.imwrite(label_filename, sem)
                    imageio.imwrite(vis_label_filename, vis_sem)

                    imageio.imwrite(label_filename, sem)
                    imageio.imwrite(vis_label_filename, vis_sem)

                    imageio.imwrite(entropy_filename, sem_uncer)
                    imageio.imwrite(vis_entropy_filename, vis_sem_uncer)

        rgbs = np.stack(rgbs, 0)
        disps = np.stack(disps, 0)
        deps = np.stack(deps, 0)
        vis_deps = np.stack(vis_deps, 0)

        if self.enable_semantic:
            sems = np.stack(sems, 0)
            vis_sems = np.stack(vis_sems, 0)
            entropys = np.stack(entropys, 0)
            vis_entropys = np.stack(vis_entropys, 0)
        else:
            sems = None
            vis_sems = None
            entropys = None
            vis_entropys = None
        return rgbs, disps, deps, vis_deps, sems, vis_sems, entropys, vis_entropys


    def render_path_for_training(self, rays, save_dir_rgbs=None, save_dir_labels = None, index=None):

        rgbs = []

        sems = []
        vis_sems = []

        to8b_np = lambda x: (255 * np.clip(x, 0, 1)).astype(np.uint8)
        to8b = lambda x: (255 * torch.clamp(x, 0, 1)).type(torch.uint8)
        logits_2_label = lambda x: torch.argmax(torch.nn.functional.softmax(x, dim=-1),dim=-1)
        logits_2_uncertainty = lambda x: torch.sum(-F.log_softmax(x, dim=-1)*F.softmax(x, dim=-1), dim=-1, keepdim=True)

        t = time.time()
        for i, c2w in enumerate(tqdm(rays)):
          if index == None:
            index = i
          else:
            index += 1
            print(i, time.time() - t)
            t = time.time()
            output_dict = self.render_rays_chunk(rays[i], self.chunk) # render rays by chunks to avoid OOM
            rgb_coarse = output_dict["rgb_coarse"]
            disp_coarse = output_dict["disp_coarse"]

            rgb = rgb_coarse
            disp = disp_coarse

            if self.enable_semantic:
                sem_label_coarse = logits_2_label(output_dict["sem_logits_coarse"])
                vis_sem_label_coarse = self.valid_colour_map[sem_label_coarse]
                # shift pred label by +1 to shift the first valid class to use the first valid colour-map instead of the void colour-map
                sem_label = sem_label_coarse
                vis_sem = vis_sem_label_coarse


            if self.N_importance > 0:
                rgb_fine = output_dict["rgb_fine"]
                rgb = rgb_fine

                if self.enable_semantic:
                    sem_label_fine = logits_2_label(output_dict["sem_logits_fine"])
                    vis_sem_label_fine = self.valid_colour_map[sem_label_fine]
                    # shift pred label by +1 to shift the first valid class to use the first valid colour-map instead of the void colour-map
                    sem_label = sem_label_fine
                    vis_sem = vis_sem_label_fine

            rgb = rgb.cpu().numpy().reshape((self.H_scaled, self.W_scaled, 3))
            rgbs.append(rgb)


            if self.enable_semantic:
                sem_label = sem_label.cpu().numpy().astype(np.uint8).reshape((self.H_scaled, self.W_scaled))
                vis_sem = vis_sem.cpu().numpy().astype(np.uint8).reshape((self.H_scaled, self.W_scaled, 3))

                sems.append(sem_label)
                vis_sems.append(vis_sem)

            if save_dir_rgbs is not None and save_dir_labels is not None:
                assert os.path.exists(save_dir_rgbs) and os.path.exists(save_dir_labels)
                rgb8 = to8b_np(rgbs[-1])


                rgb_filename = os.path.join(save_dir_rgbs, 'rgb_{:03d}.png'.format(index))


                imageio.imwrite(rgb_filename, rgb8)

                if self.enable_semantic:
                    #label_filename = os.path.join(save_dir_labels, 'label_{:03d}.png'.format(index))
                    vis_label_filename = os.path.join(save_dir_labels, 'vis_label_{:03d}.png'.format(i))
                    vis_sem = vis_sems[-1]
                    sem = sems[-1]
                    imageio.imwrite(vis_label_filename, vis_sem)
                    #imageio.imwrite(label_filename, sem)


        rgbs = np.stack(rgbs, 0)

        if self.enable_semantic:
            vis_sems = np.stack(vis_sems, 0)
            sems = np.stack(sems, 0)
        else:
            vis_sems = None
            sems = None

        return rgbs, vis_sems, sems

In [ ]:
config = {
    "experiment":{
        "width":320,
        "height":240,
        "convention":"opengl",
        "save_dir":"/content/",
        "enable_semantic":True,
        "enable_depth":False,
        "endpoint_feat":False,
    },
    "model":{
        "netdepth": 8,
        "netwidth": 256,
        "netdepth_fine": 8,
        "netwidth_fine": 256,
        "chunk": 1024*64,  # number of rays processed in parallel, decrease if running out of memory
        "netchunk": 1024*64,  # number of pts sent through network in parallel, decrease if running out of memory
    },
    "render":{
        "N_rays": 32*32*1,  # average number of rays sampled from each sample within a batch
        "N_samples": 64,  # Number of different times to sample along each ray.
        "N_importance": 128,  # Number of additional fine samples per ray
        "perturb": 1,
        "use_viewdirs": True,
        "multires": 10,  # log2 of max freq for positional encoding (3D location)'
        "multires_views": 4,  # 'log2 of max freq for positional encoding (2D direction)'
        "raw_noise_std": 1,  # 'std dev of noise added to regularize sigma_a output, 1e0 recommended')
        "test_viz_factor": 1,  # down scaling factor when rendering test and training images
        "no_batching": True,  # True-sample random pixels from random images; False-sample from all random pixels from all images
        "depth_range": [0.1, 10.0],
        "white_bkgd": False  # set to render synthetic data on a white bkgd (always use for dvoxels)
    },
    "train":{
        "lrate": 5e-4,
        "lrate_decay": 250e3,
        "N_iters": 200000,
        "wgt_sem": 4e-2,
    },
    "logging":{
        "step_log_print": 100000,  # 'frequency of console print'
        "step_log_tfb": 500,
        "step_save_ckpt": 100000,
        "step_val": 100000, # frequency of rendering on unseen data
        "step_vis_train": 100000,
    }
}

In [ ]:
ssr_trainer = SSRTrainer(config)

ssr_trainer.set_params_replica()
ssr_trainer.prepare_data_replica(dt)

# Create nerf model, init optimizer
ssr_trainer.create_ssr()
# Create rays in world coordinates
ssr_trainer.init_rays()

start = 0

N_iters = int(float(config["train"]["N_iters"])) + 1
global_step = start

tensor([ 0, 10, 14, 17, 20, 22, 29, 31, 37, 40, 47, 56, 76, 78, 80, 82, 83, 87,
        92, 93, 95, 97], dtype=torch.uint8)
Number of semantic classes:  21
prepare rays


/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


prepare rays
prepare rays


In [ ]:
ssr_trainer.step(0)

torch.Size([65536, 24])
tensor([[-0.0451, -0.0659,  0.0123,  ..., -0.0621,  0.0310,  0.0139],
        [-0.0498, -0.0633,  0.0069,  ..., -0.0654,  0.0317,  0.0088],
        [-0.0518, -0.0612,  0.0129,  ..., -0.0680,  0.0326,  0.0077],
        ...,
        [-0.0525, -0.0670,  0.0142,  ..., -0.0568,  0.0311,  0.0146],
        [-0.0517, -0.0655,  0.0099,  ..., -0.0565,  0.0281,  0.0216],
        [-0.0530, -0.0644,  0.0123,  ..., -0.0588,  0.0307,  0.0175]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
torch.Size([65536, 24])
tensor([[-0.0549,  0.0562, -0.0443,  ..., -0.0514, -0.0195,  0.0328],
        [-0.0577,  0.0571, -0.0445,  ..., -0.0509, -0.0215,  0.0332],
        [-0.0619,  0.0573, -0.0449,  ..., -0.0500, -0.0212,  0.0292],
        ...,
        [-0.0541,  0.0495, -0.0502,  ..., -0.0511, -0.0187,  0.0314],
        [-0.0560,  0.0496, -0.0519,  ..., -0.0531, -0.0229,  0.0287],
        [-0.0540,  0.0521, -0.0498,  ..., -0.0512, -0.0192,  0.0302]],
       device='cuda:0', grad_fn=<A

In [ ]:
##########################
print('Begin')
#####  Training loop  #####
tk0 = tqdm(range(start, N_iters), position=0, leave=True)
for i in tk0:

    time0 = time.time()
    ssr_trainer.step(global_step)

    dt = time.time()-time0
    #print()
    tk0.set_postfix_str(f"Time per step is :{dt}")
    #print("Time per step is :", dt)
    global_step += 1


print('done')

Begin


  0%|          | 2/200001 [00:00<11:36:11,  4.79it/s, Time per step is :0.11552309989929199]

Saved checkpoints at /content/checkpoints/000000.ckpt
[TRAIN] Iter: 0 Loss: 0.45006178958679965 rgb_total_loss: 0.20887053652073673, rgb_coarse: 0.10506865545781408, rgb_fine: 0.10380188106292265, sem_total_loss: 6.029781341552734, weighted_sem_total_loss: 0.2411912536621094, semantic_loss: 3.0134623050689697, semantic_fine: 3.0163187980651855, PSNR_coarse: 9.78526782989502, PSNR_fine: 9.837947845458984


 50%|████▉     | 100000/200001 [3:20:13<3:20:11,  8.33it/s, Time per step is :0.11794281005859375]

Saved checkpoints at /content/checkpoints/100000.ckpt
 180 train images


...

179 5.320770263671875



100%|██████████| 180/180 [15:57<00:00,  5.32s/it]


Saved test set


<ipython-input-5-101c02d5b26b>:283: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.transpose(conf_mat) / conf_mat.astype(np.float).sum(axis=1))
100%|██████████| 200001/200001 [7:45:16<00:00,  7.16it/s, Time per step is :1925.4003381729126] 

[TRAIN] Iter: 200000 Loss: 0.0029975029651843287 rgb_total_loss: 0.002094848303853003, rgb_coarse: 0.0015656158398832734, rgb_fine: 0.0005292324639697295, sem_total_loss: 0.02256636694073677, weighted_sem_total_loss: 0.0009026546776294709, semantic_loss: 0.01826232485473156, semantic_fine: 0.0043040430173277855, PSNR_coarse: 28.053146362304688, PSNR_fine: 32.76353454589844
done


In [ ]:
def generate_camera_poses(center, radius, num_poses=180):
  camera_poses = []
  for _ in range(num_poses):
    # Sample random angles within the hemisphere
    theta = np.random.uniform(0, np.pi / 2)  # Polar angle
    phi = np.random.uniform(0, 2 * np.pi)    # Azimuthal angle

    # Convert spherical coordinates to Cartesian coordinates
    x = radius * np.sin(theta) * np.cos(phi)
    y = radius * np.sin(theta) * np.sin(phi)
    z = radius * np.cos(theta)

    # Create a camera pose matrix (4x4) using look-at transformation
    # You may choose a target point or origin as the point to look at
    target_point = np.array(center)  # Change this as needed
    camera_position = np.array([x, y, z])
    forward = target_point - camera_position
    forward /= np.linalg.norm(forward)
    right = np.cross([0, 1, 0], forward)
    right /= np.linalg.norm(right)
    up = np.cross(forward, right)
    up /= np.linalg.norm(up)

    pose_matrix = np.eye(4)
    pose_matrix[:3, :3] = np.vstack((right, up, -forward)).T
    pose_matrix[:3, 3] = camera_position

    camera_poses.append(pose_matrix)
  return camera_poses

In [ ]:
def generate_circle_around_center(center, radius, theta= 1.0, num_poses=180):
  camera_poses = []
  phi = np.linspace(0,2*np.pi, num_poses)
  print("Camera_poses: ",num_poses)
  for i in range(num_poses):
    # Convert spherical coordinates to Cartesian coordinates
    x = radius * np.sin(theta) * np.cos(phi[i])
    y = radius * np.sin(theta) * np.sin(phi[i])
    z = radius * np.cos(theta)

    # Create a camera pose matrix (4x4) using look-at transformation
    # You may choose a target point or origin as the point to look at
    target_point = np.array(center)  # Change this as needed
    camera_position = np.array([x, y, z])
    forward = target_point - camera_position
    forward /= np.linalg.norm(forward)
    right = np.cross([0, 1, 0], forward)
    right /= np.linalg.norm(right)
    up = np.cross(forward, right)
    up /= np.linalg.norm(up)

    pose_matrix = np.eye(4)
    pose_matrix[:3, :3] = np.vstack((right, up, -forward)).T
    pose_matrix[:3, 3] = camera_position

    camera_poses.append(pose_matrix)
  return camera_poses

In [ ]:
generate_camera_poses([0,0,0], 1, num_poses=2)

[array([[-9.99490216e-01, -6.68189940e-04,  3.19196072e-02,
          3.19196072e-02],
        [ 0.00000000e+00,  9.99780966e-01,  2.09289412e-02,
          2.09289412e-02],
        [ 3.19266003e-02, -2.09182720e-02,  9.99271294e-01,
          9.99271294e-01],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00]]),
 array([[-9.99997437e-01,  3.82021652e-04,  2.23149672e-03,
          2.23149672e-03],
        [ 0.00000000e+00,  9.85660527e-01, -1.68740406e-01,
         -1.68740406e-01],
        [ 2.26396072e-03,  1.68739974e-01,  9.85658001e-01,
          9.85658001e-01],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00]])]

In [ ]:
ssr_trainer = SSRTrainer(config)

ssr_trainer.set_params_replica()
ssr_trainer.prepare_data_replica(dt)
# Create nerf model, init optimizer
ssr_trainer.create_ssr()
# Create rays in world coordinates
ssr_trainer.new_viewpoints_rays(180)

ssr_trainer.new_paths('/content/200000.ckpt')

tensor([ 0, 10, 14, 17, 20, 22, 29, 31, 37, 40, 47, 56, 76, 78, 80, 82, 83, 87,
        92, 93, 95, 97], dtype=torch.uint8)
Number of semantic classes:  21
Camera_poses:  180
prepare rays
 180 test images


  0%|          | 0/180 [00:00<?, ?it/s]

1 0.0033674240112304688


  1%|          | 2/180 [00:03<05:45,  1.94s/it]

2 3.887105941772461


...

178 3.949735641479492


 99%|█████████▉| 179/180 [11:39<00:03,  3.93s/it]

179 3.931527614593506


100%|██████████| 180/180 [11:42<00:00,  3.91s/it]


(179, 240, 320, 3)
Saved test set


In [ ]:
!zip -r training_dt.zip /content/training_cnn

Output streaming troncato alle ultime 5000 righe.
updating: content/training_cnn/label/label_164.png (stored 0%)
updating: content/training_cnn/label/label_2348.png (deflated 0%)
updating: content/training_cnn/label/label_933.png (stored 0%)
updating: content/training_cnn/label/label_2524.png (stored 0%)
updating: content/training_cnn/label/label_2987.png (stored 0%)
updating: content/training_cnn/label/label_1566.png (stored 0%)
updating: content/training_cnn/label/label_405.png (stored 0%)
updating: content/training_cnn/label/label_2620.png (stored 0%)
updating: content/training_cnn/label/label_2375.png (stored 0%)
updating: content/training_cnn/label/label_1157.png (stored 0%)
updating: content/training_cnn/label/label_1303.png (stored 0%)
updating: content/training_cnn/label/label_2984.png (stored 0%)
updating: content/training_cnn/label/label_945.png (stored 0%)
updating: content/training_cnn/label/label_073.png (stored 0%)
updating: content/training_cnn/label/label_2507.png (stor